<center>
<h1 style="font-size: 50px; font-weight: bold; color:sandybrown">OC SWITRS Data Processing</h1>

<div style="font-size: 40px; font-weight: bold; color: sandybrown">Part 02: ArcGIS and Spatial Geoprocessing</div>
<div style="font-size: 30px; font-weight: bold; color: sandybrown">v.5, August 2024</div>
</center>

---

## <font color="orangered">**Referencing Libraries and Initialization**</font>

### <font color="lime">**Preliminaries**</font>

Instantiating python libraries for the project

In [2]:
# Import Python libraries
import os, json, pytz, math, arcpy, arcgis
from datetime import date, time, datetime, timedelta, tzinfo, timezone
from tqdm.notebook import trange, tqdm, tqdm_notebook
import pandas as pd
import numpy as np
from arcpy import metadata as md

# important as it "enhances" Pandas by importing these classes (from ArcGIS API for Python)
from arcgis.features import GeoAccessor, GeoSeriesAccessor


### <font color="lime">**Project and Workspace Variables**</font>

Define and maintain project, workspace, ArcGIS, and data-related variables.

Project and ArcGIS Pro project paths

In [3]:
# Environment variables for OneDrive path
#oneDrivePath = os.getenv("OneDriveCommercial")

# OC SWITRS project path
#projectPath = os.path.join(oneDrivePath, "Projects", "OCTraffic", "OCSWITRS")
projectPath = os.path.join(r"I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS")

# OC SWITRS ArcGIS Pro project path
agpPath = os.path.join(projectPath, "AGPSWITRS")


ArcGIS Pro-related paths

In [4]:
# ArcGIS Pro project name and path
aprx = "AGPSWITRS.aprx"
aprxPath = os.path.join(agpPath, aprx)

# ArcGIS Pro project geodatabase and path
gdb = "AGPSWITRS.gdb"
gdbPath = os.path.join(agpPath, gdb)

# Current ArcGIS workspace (arcpy)
arcpy.env.workspace = gdbPath
workspace = arcpy.env.workspace

# Enable overwriting existing outputs
arcpy.env.overwriteOutput = True


Project folder paths

In [5]:
# Raw data folder path
rawDataPath = os.path.join(projectPath, "RawData")

# Layers folder path
layersPath = os.path.join(projectPath, "Layers")

# Notebooks folder path
notebooksPath = os.path.join(projectPath, "Notebooks")

# Supporting data path on the project geodatabase (feature directory)
supportingDataPath = os.path.join(workspace, "SupportingData")


Data folder paths and contents

The most current raw data files cover the periods from 01/01/2013 to 06/30/2024. The data files are in CSV format and are stored in the data folder, after downloaded from the SWITRS Database (https://tims.berkeley.edu/tools/query/summary.php). Date variables are defined below.

In [6]:
# add the date 01/01/2013 to a new python datetime object named 'rawDateStart'
rawDateStart = datetime(2013, 1, 1)

# add the date 06/30/2024 to a new python datetime object named 'rawDateEnd'
rawDateEnd = datetime(2024, 6, 30)


In [7]:
# Paths to raw data (crashes, parties, victims)
#rawCrashesPath = os.path.join(rawDataPath, "Crashes.csv")
#rawPartiesPath = os.path.join(rawDataPath, "Parties.csv")
#rawVictimsPath = os.path.join(rawDataPath, "Victims.csv")
rawCrashesPath = os.path.join(rawDataPath, "Crashes_"+rawDateStart.strftime("%Y%m%d")+"_"+rawDateEnd.strftime("%Y%m%d")+".csv")
rawPartiesPath = os.path.join(rawDataPath, "Parties_"+rawDateStart.strftime("%Y%m%d")+"_"+rawDateEnd.strftime("%Y%m%d")+".csv")
rawVictimsPath = os.path.join(rawDataPath, "Victims_"+rawDateStart.strftime("%Y%m%d")+"_"+rawDateEnd.strftime("%Y%m%d")+".csv")

# Path to JSON codebook
codebookPath = os.path.join(rawDataPath, "codebook.json")

# Paths to supporting data (feature classes)
boundariesPath = os.path.join(supportingDataPath, "OCSWITRS_Boundaries")
citiesPath = os.path.join(supportingDataPath, "OCSWITRS_Cities")
roadsPath = os.path.join(supportingDataPath, "OCSWITRS_Roads")
lookupPath = os.path.join(supportingDataPath, "OCSWITRS_Lookup")

# Paths to saved dataframe pickles (crashes, parties, victims, collisions)
dfCrashesPickle = os.path.join(rawDataPath, "dfCrashes.pkl")
dfPartiesPickle = os.path.join(rawDataPath, "dfParties.pkl")
dfVictimsPickle = os.path.join(rawDataPath, "dfVictims.pkl")
dfCollisionsPickle = os.path.join(rawDataPath, "dfCollisions.pkl")

# Paths to saved excel files (crashes, parties, victims, collisions) ready to be imported to ArcGIS Pro
dfCrashesExcel = os.path.join(rawDataPath, "dfCrashes.xlsx")
dfPartiesExcel = os.path.join(rawDataPath, "dfParties.xlsx")
dfVictimsExcel = os.path.join(rawDataPath, "dfVictims.xlsx")
dfCollisionsExcel = os.path.join(rawDataPath, "dfCollisions.xlsx")


Display all information about paths, folders, and data files

In [9]:
# Display all information
print("Key Project Information")
print(f"\n\t- Name {aprx}\n\t- Path: {aprxPath}\n\t- Project Path: {projectPath}\n\t- Workspace: {workspace}\n\t- Geodatabase: {gdb}\n\t- Geodatabase Path: {gdbPath}")
print("\nProject Directories:")
print(f"\n\t- Raw Data: {rawDataPath}\n\t- Layers: {layersPath}\n\t- Notebooks: {notebooksPath}\n\t- Supporting Data: {supportingDataPath}")
print("\nSupporting Feature Classes:")
print(f"\n\t- Boundaries: {boundariesPath}\n\t- Cities: {citiesPath}\n\t- Roads: {roadsPath}\n\t- Lookup: {lookupPath}")
print("\nOther Supporting Data")
print(f"\n\t- Codebook: {codebookPath}\n\t- Raw Data Files:\n\t\t- Crashes: {rawCrashesPath}\n\t\t- Parties: {rawPartiesPath}\n\t\t- Victims: {rawVictimsPath}")
print(f"\n\t- Saved datafram Pickles:\n\t\t- Crashes: {dfCrashesPickle}\n\t\t- Parties: {dfPartiesPickle}\n\t\t- Victims: {dfVictimsPickle}\n\t\t- Collisions: {dfCollisionsPickle}")
print(f"\n\t- Saved Excel dataframes:\n\t\t- Crashes: {dfCrashesExcel}\n\t\t- Parties: {dfPartiesExcel}\n\t\t- Victims: {dfVictimsExcel}\n\t\t- Collisions: {dfCollisionsExcel}")


Key Project Information

	- Name AGPSWITRS.aprx
	- Path: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.aprx
	- Project Path: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS
	- Workspace: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb
	- Geodatabase: AGPSWITRS.gdb
	- Geodatabase Path: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb

Project Directories:

	- Raw Data: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\RawData
	- Layers: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\Layers
	- Notebooks: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\Notebooks
	- Supporting Data: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\SupportingData

Supporting Feature Classes:

	- Boundaries: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\SupportingData\OCSWITRS_Boundaries
	- Cities: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\SupportingData

Define ArcGIS Pro project and map list

In [10]:
# Current ArcGIS Pro project
aprx = arcpy.mp.ArcGISProject(aprxPath)

# Current ArcGIS Pro project map
map = aprx.listMaps()[0]


## <font color="orangered">**Functions and Definitions**</font>

The definitions include:
- Dictionary Definitions (JSON)

This session will focus on the following functions:
- Date and time: `update_datetime(purpose)`
- Clean up project data: `clean_project_data(aprx, map)`
- Field type transformations: `agp_type(myType)`
- Field mappings: `get_field_mappings(myCodebook, myFieldlist, myPath)`
- Update and renumbering codebook: `update_codebook(myPath)`
- Create codebook dataframe: `create_codebook_dataframe(myCodebook)`
- Modify field types: `modify_agp_types(myCodebook, myFeatureClass)`


### <font color="lime">**Dictionary Definitions (JSON)**</font>

Defining dictionary codebook for field aliases for all three levels (crashes, parties, and victims).

The fields for each codebook contains:

1. Variable name (*key*): the JSON dictionary key for each of the variables.
2. Field alias (*alias*): the arcgis attribute field aliases for the variables.
3. Description (*desc*): the description of the variable used in the domain description definition.
4. Type (*type*): the arcgis attribute table field type.
5. Domain (*dbin*): Y/N whether the variable has a domain or not.
6. Domain Details (*domain*): If *dbin* above is Y, then this dictionary (and subsequent child entries) detail the domain attributes.

The variables and domain definitions come from the [SWIRTS Codebook Link](https://tims.berkeley.edu/help/SWITRS.php#Crash_Level)

The JSON codebooks are saved in the RawData folder of this project. Each of the codebook path can be called using the codebookPath variable from the *Preliminaries* session of this notebook.

### <font color="gold">**Function: date and time**</font>

Setting-up a function to update times, and display start and end times for each operation. Options for `purpose` include:
- *'default'*: last update with full date and time
- *'data'*: last data update with date only
- *'start'*: start time for each operation with full date and time
- *'end'*: end time for each operation with full date and time
- *'today'*: today's date with full date and time
- *'save'*: save time with full date and time
- *'load'*: load time with full date and time

In [11]:
def update_datetime(purpose="default"):
    """Function updating the date and time of the command being executed.
        
    Args:
        purpose (str, optional): The purpose of the execution. Defaults to "default". Other options include "data", "start", "end", "today", "save", "load".
            
    Returns:
        print (str): The date and time of the command being executed
    """
        
    global today, lastUpdateDate, lastUpdateDatetime, startTime, endTime, timeZone
        
    # Define time and date variables
    timeZone = pytz.timezone("US/Pacific")
    today = datetime.now(timeZone)
    lastUpdateDate = today.strftime("%B %d, %Y")
    lastUpdateDatetime = today.strftime("%A %B %d, %Y, %I:%M %p (%Z)")
        
    # Match the purpose of the function (user input)
    match purpose:
        case "default":
            print(f"Last update: {lastUpdateDatetime}")
        case "data":
            print(f"Last data update: {lastUpdateDate}")
        case "start":
            startTime = datetime.now(timeZone)
            print(f"Start: {lastUpdateDatetime}")
        case "end":
            endTime = datetime.now(timeZone)
            delta = endTime - startTime
            elapsed = delta - timedelta(microseconds=delta.microseconds)
            print(f"End: {lastUpdateDatetime}. Elapsed time: {elapsed}")
        case "today":
            print(f"Today's date: {lastUpdateDatetime}")
        case "save":
            print(f"Project saved on: {lastUpdateDatetime}")
        case "load":
            print(f"Project loaded on: {lastUpdateDatetime}")
            

### <font color="gold">**Function: clean project data**</font>

In this function we will be cleaning up the project data by removing all the tables and feature classes from the geodatabase, along with all the layers from the map.

> <font color="orange">**Note:**</font> Use this function 'as needed' usually in the beginning of running this notebook and re-running the code.

In [12]:
def clean_project_data(aprx, map):
    """Function to remove and clean all data from the geodatabase and the map of the ArcGIS Pro project.
    
    Args:
        aprx (arcpy.mp.ArcGISProject): The current ArcGIS Pro project
        map (arcpy.mp.Map): The current map in the ArcGIS Pro project
        
    Returns:
        None: The function does not return anything
    """
    
    update_datetime("start")
    
    # Remove all tables from the geodatabase
    for table in arcpy.ListTables():
        print(f"Removing table: {table}")
        arcpy.Delete_management(table)
        print(f"Removing table: {table}")
        print(arcpy.GetMessages())
    
    # Remove all feature classes from the geodatabase
    for feature in arcpy.ListFeatureClasses():
        print(f"Removing feature class: {feature}")
        arcpy.Delete_management(feature)
        print(f"Removing feature class: {feature}")
        print(arcpy.GetMessages())
    
    # Remove all layers from the map that have "OCSWITRS" in the name
    for layer in map.listLayers():
        if "OCSWITRS" in layer.name:
            print(f"Removing layer: {layer.name}")
            map.removeLayer(layer)
            print(f"Removing layer: {layer.name}")
            print(arcpy.GetMessages())

    # Save the project
    print("Saving the project")
    aprx.save()
        
    update_datetime("end")
    

### <font color="gold">**Function: Field Type Transformationss**</font>

In this function we will be changing codebook field types to match the ones genrated in geodatabase tables and feature classes.

Before using the codebook, we need to update the field types to match the ones on the table, using the following function, that reads the fied types from the tables or feature classes and changes (if necessary) the field types in the codebook.

In [13]:
def agp_type(myType):
    """Function mapping ArcGIS feature class types to codebook field types.

    Args:
        myType (str): field type from data
        
    Returns:
        str: ArcGIS feature class type
    """
    
    # Match codebook column and return the appropriate ArcGIS feature class type
    match myType:
        case "OID":
            return "ID"
        case "GlobalID":
            return "GlobalID"
        case "GUID":
            return "GUID"
        case "Geometry":
            return "GEOMETRY"
        case "Integer":
            return "LONG"
        case "SmallInteger":
            return "SHORT"
        case "BigInteger":
            return "BIG"
        case "Double":
            return "DOUBLE"
        case "Date":
            return "DATE"
        case "DateOnly":
            return "DATE"
        case "String":
            return "TEXT"


### <font color="gold">**Function: get field mappings**</font>

This function generates field mapping lists for operations that require field mapping. The function takes three arguments: `myCodebook`, `myFieldlist`, and `myPath`.

In [14]:
def get_field_mappings(myCodebook, myFieldList, myPath):
    """Function generating field mapping lists of fields used in ArcGIS geoprocessing tasks.

    Args:
        myCodebook (dict): the project's JSON dictionary containing the codebook of all the data variables
        myFieldList (list): the list of fields for which the field mapping is performed
        myPath (path): the path to the feature class being processed.
        
    Returns:
        fieldMappings (arcpy.FieldMappings): the field mapping object used in ArcGIS geoprocessing tasks
    """
    
    # Get a list of field names from the user-submitted field list
    myFieldNames = list(myField.name for myField in myFieldList)
    
    # Create an empty list to hold the field mappings
    myMappingList = []
    
    # Iterate the keys of the codebook dictionary
    for key in list(myCodebook.keys()):        
        # Check if the key is in the list of field names
        if key in myFieldNames:
            # Obtain the index location of the key field in the dataset
            myIndex = myFieldNames.index(key)
            # Obtain the field type of the field
            myType = agp_type(myFieldList[myIndex].type)
            if myType in ["ID", "GlobalID", "GUID", "GEOMETRY", "LONG", "SHORT", "BIG", "DOUBLE"]:
                s = "-1"
            elif myType in ["TEXT", "DATE", "DATEONLY"]:
                s = "#"
                
            # Create a field mapping object string for each of the fields in the dataset
            mapString = f'{myFieldList[myIndex].name} "{myFieldList[myIndex].aliasName}" {myFieldList[myIndex].editable} {myFieldList[myIndex].isNullable} {myFieldList[myIndex].required} {myFieldList[myIndex].length} {myType} {myFieldList[myIndex].precision} {myFieldList[myIndex].scale},First,#,{myPath},{myFieldList[myIndex].name},"{myFieldList[myIndex].aliasName}",{s},{s}'
            
            # Append the string to the list of field mappings
            myMappingList.append(mapString)
            
    # Join the list of field mappings into a single string (separated by semicolons)
    return ";".join(myMappingList)
                

### <font color="gold">**Function: update codebook**</font>

This function reads a JSON file from a path, updates the numbering and order of the fields, and returns a functional codebook dictionary for the project.

In [15]:
def update_codebook(myPath):
    """Function to update the JSON codebook file
        
    Args:
        myPath (str): path to the JSON codebook file
            
    Returns:
        dictData (dict): updated JSON dictionary data
    """
        
    # Load the JSON file from directory
    print("Opening dictionary from file...")
    with open(myPath, "r") as jsonFile:
        # Load the JSON data into dictionary
        dictData = json.load(jsonFile)
        
    # start the counter
    counter = 1
        
    # Loop through the dictionary and update the "no" key
    print("Renumbering the dictionary...")
        
    for key in dictData:
        if "no" in dictData[key]:
            dictData[key]["no"] = counter
            # increment the counter
            counter += 1
        
    # Reopen the JSON data file for writing
    print("Writing the updated dictionary to file...")
    with open(myPath, "w") as jsonFile:
        # Write the updated dictionary to the file
        json.dump(dictData, jsonFile, indent=4)
        
    print(f"Total records: {counter - 1}\nOutput file: {myPath}")
        
    return dictData


### <font color="gold">**Function: create codebook dataframe**</font>

Creating and displaying a Pandas dataframe from the codebook JSON dictionary

In [16]:
def create_codebook_dataframe(myCodebook):
    """Function creating a Pandas dataframe from the project's JSON codebook dictionary.

    Args:
        myCodebook (dict): the project's JSON codebook dictionary
        
    Returns:
        pd.dataframe (dataFrame): the Pandas dataframe of the codebook
    """
    
    # Create a new, empty pandas dataframe
    mydf = {}
    
    # Add the codebook keys as column fields, and the codebook values as rows for the dataframe
    mydf["ORDER"] = [myCodebook[field]["no"] for field in myCodebook]
    mydf["COLUMN"] = [field for field in myCodebook]
    mydf["NAME"] = [myCodebook[field]["alias"] for field in myCodebook]
    mydf["DESCRIPTION"] = [myCodebook[field]["desc"] for field in myCodebook]
    mydf["LEVEL"] = [myCodebook[field]["level"] for field in myCodebook]
    mydf["TYPE"] = [myCodebook[field]["type"] for field in myCodebook]
    mydf["CATEGORY"] = [myCodebook[field]["cat"] for field in myCodebook]
    mydf["DOMAIN"] = [myCodebook[field]["dbin"] for field in myCodebook]
    
    return pd.DataFrame(mydf)


### <font color="gold">**Function: modify field types**</font>

This function is modifying the field types on the codebook, based on the fields on a feature class field list

In [17]:
def modify_agp_types(myCodebook, myFeatureClass):
    """Function modifying field types on the codebook, based on the fields in the feature class.

    Args:
        myCodebook (dict): the codebook JSON dictionary of the project
        myFeatureClass (str): the name of the ArcGIS feature class to be used for
        
    Returns:
        myCodebook (dict): the updated codebook JSON dictionary
    """
    
    # Iterate feature class columns and select each column by name
    for field in list(myFeatureClass):
        # Check first if the feature class field exists in the codebook keys
        if field in list(myCodebook.keys()):
            # Obtain the codebook field type from the codebook dictionary
            myCodebookType = myCodebook[field]["type"]
            # Get the corresponding feature class type based on the function agp_type() defined above
            myDataframeType = agp_type(myDataframeType)
            
            # Check to see if these two types don't match
            if myCodebookType != myDataframeType:
                print(f"\t{field}: {myCodebookType} -> {myDataframeType}")
                # Update the codebook field type to match the feature class field type
                myCodebook[field]["type"] = myDataframeType
                
    # Return the updated codebook dictionary
    return myCodebook


## <font color="orangered">**Data Procesing Operations**</font>

### <font color="lime">**Data Prepparation**</font>

If needed clean up the project data

> <font color="orange">⚠️ **Note:**</font> Use this function 'as needed' usually in the beginning of running this notebook and re-running the code.

In [18]:
# Clean the project data
clean_project_data(aprx, map)


Start: Wednesday November 13, 2024, 06:55 AM (PST)
Saving the project
End: Wednesday November 13, 2024, 06:55 AM (PST). Elapsed time: 0:00:00


Run the function `update_codebook(myPath)` to update the codebook for the project for the first time

In [19]:
codebook = update_codebook(codebookPath)


Opening dictionary from file...
Renumbering the dictionary...
Writing the updated dictionary to file...
Total records: 145
Output file: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\RawData\codebook.json


Create the codebook dataframe and display the table for inspection

In [20]:
dfCodebook = create_codebook_dataframe(codebook)


Display the codebook dataframe with all rows visible

In [21]:
# Display codebook dataframe
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(dfCodebook)


,ORDER,COLUMN,NAME,DESCRIPTION,LEVEL,TYPE,CATEGORY,DOMAIN
0,1,OBJECTID,Object ID,ESRI Object ID,GIS,OID,Unique Identifier,N
1,2,GlobalID,Global ID,ESRI Global ID string identifier,GIS,GlobalID,Unique Identifier,N
2,3,GUID,GUID Related Table Identifier,ESRI GUID to be used in merging related tables,GIS,Guid,Unique Identifier,N
3,4,CASE_ID,Case ID,Unique identifier of the crash case,Crash,LONG,Unique Identifier,N
4,5,CID,Crash ID,Unique identifier of the crash case,Crash,TEXT,Unique Identifier,N
5,6,PID,Party ID,Unique identifier for the party case,Party,TEXT,Unique Identifier,N
6,7,VID,Victim ID,Unique identifier of the victim case,Victim,TEXT,Unique Identifier,N
7,8,PARTY_NUMBER,Party Number,A number that together with the CASE_ID unique...,Party,LONG,Unique Identifier,N
8,9,VICTIM_NUMBER,Victim Number,The unique identifier of the victim,Victim,LONG,Unique Identifier,N
9,10,CRASHES_INDICATOR,Crashes Binary Indicator,"Binary flag, 1 if the record is a crash, 0 if not",Crash,LONG,Binary,N


### <font color="lime">**Importing Excel data into geodatabase tables**</font>

Importing the excel files into geodatabase tables in ArcGIS Pro

#### <font color="deepskyblue">Crashes Table</font>

Importing the Crashes excel file into the ArcGIS Pro project as a geodatabase table

In [22]:
update_datetime("start")

print("Converting Excel to Table: CrashesRaw")
# Importing the Crashes excel file into an ArCGIS Pro project table
arcpy.conversion.ExcelToTable(
    Input_Excel_File = dfCrashesExcel,
    Output_Table = "CrashesRaw",
    field_names_row = 1,
    cell_range = ""
)
print(arcpy.GetMessages())

update_datetime("end")


Start: Wednesday November 13, 2024, 06:55 AM (PST)
Converting Excel to Table: CrashesRaw
Start Time: Wednesday, November 13, 2024 6:55:39 AM
Succeeded at Wednesday, November 13, 2024 7:04:08 AM (Elapsed Time: 8 minutes 29 seconds)
End: Wednesday November 13, 2024, 07:04 AM (PST). Elapsed time: 0:08:30


#### <font color="deepskyblue">Parties Table</font>

Importing the Parties excel table into the ArcGIS Pro project as a geodatabase table

In [23]:
update_datetime("start")

print("Converting Excel to Table: PartiesRaw")
# Importing the Parties excel file into an ArCGIS Pro project table
arcpy.conversion.ExcelToTable(
    Input_Excel_File = dfPartiesExcel,
    Output_Table = "PartiesRaw",
    field_names_row = 1,
    cell_range = ""
)
print(arcpy.GetMessages())

update_datetime("end")


Start: Wednesday November 13, 2024, 07:04 AM (PST)
Converting Excel to Table: PartiesRaw
Start Time: Wednesday, November 13, 2024 7:04:08 AM
Succeeded at Wednesday, November 13, 2024 7:29:21 AM (Elapsed Time: 25 minutes 12 seconds)
End: Wednesday November 13, 2024, 07:29 AM (PST). Elapsed time: 0:25:13


#### <font color="deepskyblue">Victims Table</font>

Importing the Victims excel table into the ArcGIS Pro project as a geodatabase table

In [24]:
update_datetime("start")

print("Convert Excel to Table: VictimsRaw")
# Importing the Victims excel file into an ArCGIS Pro project table
arcpy.conversion.ExcelToTable(
    Input_Excel_File = dfVictimsExcel,
    Output_Table = "VictimsRaw",
    field_names_row = 1,
    cell_range = ""
)
print(arcpy.GetMessages())

update_datetime("end")


Start: Wednesday November 13, 2024, 07:29 AM (PST)
Convert Excel to Table: VictimsRaw
Start Time: Wednesday, November 13, 2024 7:29:21 AM
Succeeded at Wednesday, November 13, 2024 7:52:24 AM (Elapsed Time: 23 minutes 2 seconds)
End: Wednesday November 13, 2024, 07:52 AM (PST). Elapsed time: 0:23:02


#### <font color="deepskyblue">Collisions Table</font>

Importing the Collisions excel table into the ArcGIS Pro project as a geodatabase table

In [25]:
update_datetime('end')

print("Converting Excel to Table: CollisionsRaw")
# Importing the Collisions excel file into an ArcGIS Pro project table
arcpy.conversion.ExcelToTable(
    Input_Excel_File = dfCollisionsExcel,
    Output_Table = "CollisionsRaw",
    field_names_row = 1,
    cell_range = ""
)
print(arcpy.GetMessages())

update_datetime("end")


End: Wednesday November 13, 2024, 07:52 AM (PST). Elapsed time: 0:23:02
Converting Excel to Table: CollisionsRaw
Start Time: Wednesday, November 13, 2024 7:52:24 AM
Succeeded at Wednesday, November 13, 2024 8:16:52 AM (Elapsed Time: 24 minutes 28 seconds)
End: Wednesday November 13, 2024, 08:16 AM (PST). Elapsed time: 0:47:31


Save the ArCGIS Pro project

In [26]:
# Save the project
aprx.save()


#### <font color="deepskyblue">Updating fields and counts</font>

Updating lists: tables, fields, and row counts from the ArcGIS Pro project geodatabase

In [27]:
# Get the list of tables in the geodatabase
gdbTables = arcpy.ListTables()

# Get each of the individual tables in the geodatabase
crashesTable = os.path.join(workspace, arcpy.ListTables("CrashesRaw")[0])
partiesTable = os.path.join(workspace, arcpy.ListTables("PartiesRaw")[0])
victimsTable = os.path.join(workspace, arcpy.ListTables("VictimsRaw")[0])
collisionsTable = os.path.join(workspace, arcpy.ListTables("CollisionsRaw")[0])

print(f"Geodatabase Tables: {gdbTables}")
print(f"\tCrashes: {crashesTable}\n\tParties: {partiesTable}\n\tVictims: {victimsTable}\n\tCollisions: {collisionsTable}")


Geodatabase Tables: ['CrashesRaw', 'PartiesRaw', 'VictimsRaw', 'CollisionsRaw']
	Crashes: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\CrashesRaw
	Parties: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\PartiesRaw
	Victims: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\VictimsRaw
	Collisions: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\CollisionsRaw


Updating the list of fields from each of the geodatabase tables

In [28]:
# Field lists for each of the geodatabase data tables
fieldsCrashes = arcpy.ListFields(crashesTable)
fieldsParties = arcpy.ListFields(partiesTable)
fieldsVictims = arcpy.ListFields(victimsTable)
fieldsCollisions = arcpy.ListFields(collisionsTable)


Count rows in each of the geodatabase tables

In [29]:
# Get the row count for each of the tables
countCrashes = int(arcpy.management.GetCount(crashesTable)[0])
countParties = int(arcpy.management.GetCount(partiesTable)[0])
countVictims = int(arcpy.management.GetCount(victimsTable)[0])
countCollisions = int(arcpy.management.GetCount(collisionsTable)[0])

print(f"Counts:\n- Crashes: {countCrashes:,}\n- Parties: {countParties:,}\n- Victims: {countVictims:,}\n- Collisions: {countCollisions:,}")


Counts:
- Crashes: 149,764
- Parties: 323,542
- Victims: 260,120
- Collisions: 260,120


Updating the mappings of the tables

In [30]:
# Mapping the fields using the function get_field_mappings() for each of the tables
mappingCrashes = get_field_mappings(codebook, fieldsCrashes, crashesTable)
mappingParties = get_field_mappings(codebook, fieldsParties, partiesTable)
mappingVictims = get_field_mappings(codebook, fieldsVictims, victimsTable)
mappingCollisions = get_field_mappings(codebook, fieldsCollisions, collisionsTable)


### <font color="lime">**Reordering and Exporting Tables**</font>

In this subsection, we will reorder the fields in the tables to match the codebook order and will remove any old fields that are not in the codebook.

#### <font color="deepskyblue">Crashes Table</font>

Reordering and exporting the Crashes table

In [31]:
update_datetime("start")

# Generate field list and field mappings for the Crashes table
currentFieldList = arcpy.ListFields(crashesTable)
currentFieldMappings = get_field_mappings(codebook, currentFieldList, crashesTable)

print("Exporting Crashes Table")
# Export the table to a new table
arcpy.conversion.ExportTable(
    in_table = crashesTable,
    out_table = os.path.join(workspace, "CrashesTable"),
    where_clause = "",
    use_field_alias_as_name = "NOT_USE_ALIAS",
    field_mapping = currentFieldMappings,
    sort_field = None
)
print(arcpy.GetMessages())

update_datetime("end")


Start: Wednesday November 13, 2024, 08:17 AM (PST)
Exporting Crashes Table
Start Time: Wednesday, November 13, 2024 8:17:39 AM
Succeeded at Wednesday, November 13, 2024 8:18:03 AM (Elapsed Time: 23.57 seconds)
End: Wednesday November 13, 2024, 08:18 AM (PST). Elapsed time: 0:00:24


#### <font color="deepskyblue">Parties Table</font>

Reordering and exporting the Parties table

In [32]:
update_datetime("start")

# Generate field list and field mappings for the Parties table
currentFieldList = arcpy.ListFields(partiesTable)
currentFieldMappings = get_field_mappings(codebook, currentFieldList, partiesTable)

print("Exporting Parties Table")
# Export the table to a new table
arcpy.conversion.ExportTable(
    in_table = partiesTable,
    out_table = os.path.join(workspace, "PartiesTable"),
    where_clause = "",
    use_field_alias_as_name = "NOT_USE_ALIAS",
    field_mapping = currentFieldMappings,
    sort_field = None
)
print(arcpy.GetMessages())

update_datetime("end")


Start: Wednesday November 13, 2024, 08:18 AM (PST)
Exporting Parties Table
Start Time: Wednesday, November 13, 2024 8:18:03 AM
Succeeded at Wednesday, November 13, 2024 8:19:08 AM (Elapsed Time: 1 minutes 4 seconds)
End: Wednesday November 13, 2024, 08:19 AM (PST). Elapsed time: 0:01:04


#### <font color="deepskyblue">Victims Table</font>

Reordering and exporting the Victims table

In [33]:
update_datetime("start")

# Generate field list and field mappings for the Victims table
currentFieldList = arcpy.ListFields(victimsTable)
currentFieldMappings = get_field_mappings(codebook, currentFieldList, victimsTable)

print("Exporting Victims Table")
# Export the table to a new table
arcpy.conversion.ExportTable(
    in_table = victimsTable,
    out_table = os.path.join(workspace, "VictimsTable"),
    where_clause = "",
    use_field_alias_as_name = "NOT_USE_ALIAS",
    field_mapping = currentFieldMappings,
    sort_field = None
)
print(arcpy.GetMessages())

update_datetime("end")


Start: Wednesday November 13, 2024, 08:19 AM (PST)
Exporting Victims Table
Start Time: Wednesday, November 13, 2024 8:19:09 AM
Succeeded at Wednesday, November 13, 2024 8:20:05 AM (Elapsed Time: 56.45 seconds)
End: Wednesday November 13, 2024, 08:20 AM (PST). Elapsed time: 0:00:57


#### <font color="deepskyblue">Collisions Table</font>

Reordering and exporting the Collisions table

In [34]:
update_datetime("start")

# Generate field list and field mappings for the Collisions table
currentFieldList = arcpy.ListFields(collisionsTable)
currentFieldMappings = get_field_mappings(codebook, currentFieldList, collisionsTable)

print("Exporting Collisions Table")
# Export the table to a new table
arcpy.conversion.ExportTable(
    in_table = collisionsTable,
    out_table = os.path.join(workspace, "CollisionsTable"),
    where_clause = "",
    use_field_alias_as_name = "NOT_USE_ALIAS",
    field_mapping = currentFieldMappings,
    sort_field = None
)
print(arcpy.GetMessages())

update_datetime("end")


Start: Wednesday November 13, 2024, 08:20 AM (PST)
Exporting Collisions Table
Start Time: Wednesday, November 13, 2024 8:20:06 AM
Succeeded at Wednesday, November 13, 2024 8:21:01 AM (Elapsed Time: 55.45 seconds)
End: Wednesday November 13, 2024, 08:21 AM (PST). Elapsed time: 0:00:56


Delete the old unordered tables from the ArcGIS Pro project

In [35]:
update_datetime("start")

print("Deleting old tables")
# Delete old tables
arcpy.management.Delete(
    in_data = f"CrashesRaw; PartiesRaw; VictimsRaw; CollisionsRaw; {crashesTable}; {partiesTable}; {victimsTable}; {collisionsTable}",
    data_type = ""
)
print(arcpy.GetMessages())

update_datetime("end")


Start: Wednesday November 13, 2024, 08:23 AM (PST)
Deleting old tables
Start Time: Wednesday, November 13, 2024 8:23:31 AM
Succeeded at Wednesday, November 13, 2024 8:23:35 AM (Elapsed Time: 3.56 seconds)
End: Wednesday November 13, 2024, 08:23 AM (PST). Elapsed time: 0:00:04


Delete old and obsolete OBJECTID fields from the tables

In [36]:
update_datetime("start")

# Iterate through the four tables and delete the OBJECTID_1 field
for table in [os.path.join(workspace, "CrashesTable"), os.path.join(workspace, "PartiesTable"), os.path.join(workspace, "VictimsTable"), os.path.join(workspace, "CollisionsTable")]:
    print(f"Deleting field OBJECTID_1 from {table}")
    arcpy.management.DeleteField(
        in_table = table,
        drop_field = "OBJECTID_1",
        method = "DELETE_FIELDS"
    )
    print(arcpy.GetMessages())

update_datetime("end")


Start: Wednesday November 13, 2024, 08:23 AM (PST)
Deleting field OBJECTID_1 from I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\CrashesTable
Start Time: Wednesday, November 13, 2024 8:23:38 AM
Succeeded at Wednesday, November 13, 2024 8:23:57 AM (Elapsed Time: 18.11 seconds)
Deleting field OBJECTID_1 from I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\PartiesTable
Start Time: Wednesday, November 13, 2024 8:23:57 AM
Succeeded at Wednesday, November 13, 2024 8:24:38 AM (Elapsed Time: 41.16 seconds)
Deleting field OBJECTID_1 from I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\VictimsTable
Start Time: Wednesday, November 13, 2024 8:24:38 AM
Succeeded at Wednesday, November 13, 2024 8:25:09 AM (Elapsed Time: 31.11 seconds)
Deleting field OBJECTID_1 from I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\CollisionsTable
Start Time: Wednesday, November 13, 2024 8:25:10 AM
Succeeded at Wednesda

### <font color="lime">**Updating Tables and Field Lists**</font>

Updating the list and contents of the python variables containing the tables

In [37]:
# Get the table list from the ArcGIS Pro project geodatabase
gdbTables = arcpy.ListTables()

# Get all the tables by name
crashesTable = os.path.join(workspace, arcpy.ListTables("CrashesTable")[0])
partiesTable = os.path.join(workspace, arcpy.ListTables("PartiesTable")[0])
victimsTable = os.path.join(workspace, arcpy.ListTables("VictimsTable")[0])
collisionsTable = os.path.join(workspace, arcpy.ListTables("CollisionsTable")[0])

print(f"Geodatabase Tables: {gdbTables}")
print(f"\tCrashes: {crashesTable}\n\tParties: {partiesTable}\n\tVictims: {victimsTable}\n\tCollisions: {collisionsTable}")


Geodatabase Tables: ['CrashesTable', 'PartiesTable', 'VictimsTable', 'CollisionsTable']
	Crashes: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\CrashesTable
	Parties: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\PartiesTable
	Victims: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\VictimsTable
	Collisions: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\CollisionsTable


Updating the field lists of the tables

In [38]:
# Field lists for each of the geodatabase data tables
fieldsCrashes = arcpy.ListFields(crashesTable)
fieldsParties = arcpy.ListFields(partiesTable)
fieldsVictims = arcpy.ListFields(victimsTable)
fieldsCollisions = arcpy.ListFields(collisionsTable)


Count rows in each of the geodatabase tables

In [39]:
# Get the row count for each of the tables
countCrashes = int(arcpy.management.GetCount(crashesTable)[0])
countParties = int(arcpy.management.GetCount(partiesTable)[0])
countVictims = int(arcpy.management.GetCount(victimsTable)[0])
countCollisions = int(arcpy.management.GetCount(collisionsTable)[0])

print(f"Counts:\n- Crashes: {countCrashes:,}\n- Parties: {countParties:,}\n- Victims: {countVictims:,}\n- Collisions: {countCollisions:,}")


Counts:
- Crashes: 149,764
- Parties: 323,542
- Victims: 260,120
- Collisions: 260,120


Updating the mappings of the tables

In [40]:
# Mapping the fields using the function get_field_mappings() for each of the tables
mappingCrashes = get_field_mappings(codebook, fieldsCrashes, crashesTable)
mappingParties = get_field_mappings(codebook, fieldsParties, partiesTable)
mappingVictims = get_field_mappings(codebook, fieldsVictims, victimsTable)
mappingCollisions = get_field_mappings(codebook, fieldsCollisions, collisionsTable)


Save the ArcGIS Pro project

In [41]:
# Save the project
aprx.save()


## <font color="orangered">**Codebook for Field Aliases**</font>

### <font color="lime">**Assigning field aliases**</font>

Creating and assigning field aliases for the geodatabase tables using the JSON codebook dictionary

#### <font color="deepskyblue">Crashes Table</font>

Field aliases for the Crashes table

In [42]:
update_datetime("start")

# Field aliases for the Crashes geodatabase table
print("Adding field aliases to the Crashes table...")
for field in fieldsCrashes:
    if field.name in list(codebook.keys()):
        print(f"\tMatch {codebook[field.name]['no']}: {field.name} ({codebook[field.name]['alias']}): {codebook[field.name]['desc']}")
        arcpy.management.AlterField(
            in_table = crashesTable,
            field = field.name,
            new_field_name = field.name,
            new_field_alias = codebook[field.name]["alias"]
        )
        #print(arcpy.GetMessages())
        
update_datetime("end")


Start: Wednesday November 13, 2024, 08:26 AM (PST)
Adding field aliases to the Crashes table...
	Match 1: OBJECTID (Object ID): ESRI Object ID
	Match 4: CASE_ID (Case ID): Unique identifier of the crash case
	Match 5: CID (Crash ID): Unique identifier of the crash case
	Match 13: CITY (City): Reported City of the crash
	Match 14: CITY_NAME (City Name): City name from locational query
	Match 15: PLACE_TYPE (Place Type): City or Unincorporated Area
	Match 16: COLLISION_DATETIME (Crash Date and Time): Full date and time string of the crash
	Match 17: COLLISION_DATE (Crash Date): The date when the crash occurred
	Match 18: COLLISION_TIME (Crash Time): The time when the crash occurred in 24h format
	Match 19: ACCIDENT_YEAR (Crash Year): The year when the crash occurred
	Match 20: COLLISION_MONTH (Crash Month): Month number from crash date
	Match 21: DAY_OF_WEEK (Day of Week): Code for the day of the week when the crash occurred
	Match 22: PROC_DATE (Processing Date): The date that the crash

#### <font color="deepskyblue">Parties Table</font>

Field aliases for the Parties table

In [43]:
update_datetime("start")

# Field aliases for the Parties geodatabase table
print("Adding field aliases to the Parties table...")
for field in fieldsParties:
    if field.name in list(codebook.keys()):
        print(f"\tMatch {codebook[field.name]['no']}: {field.name} ({codebook[field.name]['alias']}): {codebook[field.name]['desc']}")
        arcpy.management.AlterField(
            in_table = partiesTable,
            field = field.name,
            new_field_name = field.name,
            new_field_alias = codebook[field.name]["alias"]
        )
        #print(arcpy.GetMessages())
        
update_datetime("end")


Start: Wednesday November 13, 2024, 08:27 AM (PST)
Adding field aliases to the Parties table...
	Match 1: OBJECTID (Object ID): ESRI Object ID
	Match 4: CASE_ID (Case ID): Unique identifier of the crash case
	Match 5: CID (Crash ID): Unique identifier of the crash case
	Match 6: PID (Party ID): Unique identifier for the party case
	Match 8: PARTY_NUMBER (Party Number): A number that together with the CASE_ID uniquely identifies a party in a crash
	Match 13: CITY (City): Reported City of the crash
	Match 14: CITY_NAME (City Name): City name from locational query
	Match 15: PLACE_TYPE (Place Type): City or Unincorporated Area
	Match 16: COLLISION_DATETIME (Crash Date and Time): Full date and time string of the crash
	Match 17: COLLISION_DATE (Crash Date): The date when the crash occurred
	Match 18: COLLISION_TIME (Crash Time): The time when the crash occurred in 24h format
	Match 19: ACCIDENT_YEAR (Crash Year): The year when the crash occurred
	Match 20: COLLISION_MONTH (Crash Month): Mo

#### <font color="deepskyblue">Victims Table</font>

Field aliases for the Victims table

In [44]:
update_datetime("start")

# Field aliases for the Victims geodatabase table
print("Adding field aliases to the Victims table...")
for field in fieldsVictims:
    if field.name in list(codebook.keys()):
        print(f"\tMatch {codebook[field.name]['no']}: {field.name} ({codebook[field.name]['alias']}): {codebook[field.name]['desc']}")
        arcpy.management.AlterField(
            in_table = victimsTable,
            field = field.name,
            new_field_name = field.name,
            new_field_alias = codebook[field.name]["alias"]
        )
        #print(arcpy.GetMessages())
        
update_datetime("end")


Start: Wednesday November 13, 2024, 08:28 AM (PST)
Adding field aliases to the Victims table...
	Match 1: OBJECTID (Object ID): ESRI Object ID
	Match 4: CASE_ID (Case ID): Unique identifier of the crash case
	Match 5: CID (Crash ID): Unique identifier of the crash case
	Match 6: PID (Party ID): Unique identifier for the party case
	Match 7: VID (Victim ID): Unique identifier of the victim case
	Match 8: PARTY_NUMBER (Party Number): A number that together with the CASE_ID uniquely identifies a party in a crash
	Match 9: VICTIM_NUMBER (Victim Number): The unique identifier of the victim
	Match 13: CITY (City): Reported City of the crash
	Match 14: CITY_NAME (City Name): City name from locational query
	Match 15: PLACE_TYPE (Place Type): City or Unincorporated Area
	Match 16: COLLISION_DATETIME (Crash Date and Time): Full date and time string of the crash
	Match 17: COLLISION_DATE (Crash Date): The date when the crash occurred
	Match 18: COLLISION_TIME (Crash Time): The time when the cras

#### <font color="deepskyblue">Collisions Table</font>

Field aliases for the Collisions table

In [45]:
update_datetime("start")

# Field aliases for the Collisions geodatabase table
print("Adding field aliases to the Collisions table...")
for field in fieldsCollisions:
    if field.name in list(codebook.keys()):
        print(f"\tMatch {codebook[field.name]['no']}: {field.name} ({codebook[field.name]['alias']}): {codebook[field.name]['desc']}")
        arcpy.management.AlterField(
            in_table = collisionsTable,
            field = field.name,
            new_field_name = field.name,
            new_field_alias = codebook[field.name]["alias"]
        )
        #print(arcpy.GetMessages())
        
update_datetime("end")


Start: Wednesday November 13, 2024, 08:29 AM (PST)
Adding field aliases to the Collisions table...
	Match 1: OBJECTID (Object ID): ESRI Object ID
	Match 4: CASE_ID (Case ID): Unique identifier of the crash case
	Match 5: CID (Crash ID): Unique identifier of the crash case
	Match 6: PID (Party ID): Unique identifier for the party case
	Match 7: VID (Victim ID): Unique identifier of the victim case
	Match 8: PARTY_NUMBER (Party Number): A number that together with the CASE_ID uniquely identifies a party in a crash
	Match 9: VICTIM_NUMBER (Victim Number): The unique identifier of the victim
	Match 13: CITY (City): Reported City of the crash
	Match 14: CITY_NAME (City Name): City name from locational query
	Match 15: PLACE_TYPE (Place Type): City or Unincorporated Area
	Match 16: COLLISION_DATETIME (Crash Date and Time): Full date and time string of the crash
	Match 17: COLLISION_DATE (Crash Date): The date when the crash occurred
	Match 18: COLLISION_TIME (Crash Time): The time when the c

Save the ArcGIS Pro project

In [46]:
# Save the project
aprx.save()


## <font color="coral">**Geodatabase Tables to Point Feature Classes**</font>

### <font color="lime">**Tables to Point Features Conversion**</font>

In this section we will convert the geodatabase tables to point feature classes.

First, we will geocode the tables to create point feature classes for the Crashes, Parties, Victims, and Collisions tables from the X and Y coordinates.

#### <font color="deepskyblue">Geocoding Crashes Table</font>

Processing Crashes table

In [47]:
update_datetime("start")

print("Creating a point feature class from the X and Y coordinates of the Crashes table")
# Crashes table to point feature
arcpy.management.XYTableToPoint(
    in_table = crashesTable,
    out_feature_class = os.path.join(workspace, "CrashesPoints"),
    x_field = "POINT_X",
    y_field = "POINT_Y",
    z_field = None,
    coordinate_system = arcpy.SpatialReference(4326)
)
print(arcpy.GetMessages())

update_datetime("end")


Start: Wednesday November 13, 2024, 08:31 AM (PST)
Creating a point feature class from the X and Y coordinates of the Crashes table
Start Time: Wednesday, November 13, 2024 8:31:10 AM
Succeeded at Wednesday, November 13, 2024 8:31:29 AM (Elapsed Time: 19.01 seconds)
End: Wednesday November 13, 2024, 08:31 AM (PST). Elapsed time: 0:00:19


#### <font color="deepskyblue">Geocoding Parties Table</font>

Processing Parties table

In [48]:
update_datetime("start")

print("Creating a point feature class from the X and Y coordinates of the Parties table")
# Parties table to point feature
arcpy.management.XYTableToPoint(
    in_table = partiesTable,
    out_feature_class = os.path.join(workspace, "PartiesPoints"),
    x_field = "POINT_X",
    y_field = "POINT_Y",
    z_field = None,
    coordinate_system = arcpy.SpatialReference(4326)
)
print(arcpy.GetMessages())

update_datetime("end")


Start: Wednesday November 13, 2024, 08:31 AM (PST)
Creating a point feature class from the X and Y coordinates of the Parties table
Start Time: Wednesday, November 13, 2024 8:31:29 AM
Succeeded at Wednesday, November 13, 2024 8:32:13 AM (Elapsed Time: 43.98 seconds)
End: Wednesday November 13, 2024, 08:32 AM (PST). Elapsed time: 0:00:44


#### <font color="deepskyblue">Geocoding Victims Table</font>


Processing Victims table

In [49]:
update_datetime("start")

print("Creating a point feature class from the X and Y coordinates of the Victims table")
# Victims table to point feature
arcpy.management.XYTableToPoint(
    in_table = victimsTable,
    out_feature_class = os.path.join(workspace, "VictimsPoints"),
    x_field = "POINT_X",
    y_field = "POINT_Y",
    z_field = None,
    coordinate_system = arcpy.SpatialReference(4326)
)
print(arcpy.GetMessages())

update_datetime("end")


Start: Wednesday November 13, 2024, 08:32 AM (PST)
Creating a point feature class from the X and Y coordinates of the Victi table
Start Time: Wednesday, November 13, 2024 8:32:14 AM
Succeeded at Wednesday, November 13, 2024 8:32:50 AM (Elapsed Time: 35.79 seconds)
End: Wednesday November 13, 2024, 08:32 AM (PST). Elapsed time: 0:00:36


#### <font color="deepskyblue">Geocoding Collisions Table</font>

Processing Collisions table

In [50]:
update_datetime("start")

print("Creating a point feature class from the X and Y coordinates of the Collisions table")
# Collisions table to point feature
arcpy.management.XYTableToPoint(
    in_table = collisionsTable,
    out_feature_class = os.path.join(workspace, "CollisionsPoints"),
    x_field = "POINT_X",
    y_field = "POINT_Y",
    z_field = None,
    coordinate_system = arcpy.SpatialReference(4326)
)
print(arcpy.GetMessages())

update_datetime("end")


Start: Wednesday November 13, 2024, 08:32 AM (PST)
Creating a point feature class from the X and Y coordinates of the Collisions table
Start Time: Wednesday, November 13, 2024 8:32:50 AM
Succeeded at Wednesday, November 13, 2024 8:33:25 AM (Elapsed Time: 35.52 seconds)
End: Wednesday November 13, 2024, 08:33 AM (PST). Elapsed time: 0:00:35


Save the ArcGIS Pro project

In [51]:
# Save the project
aprx.save()


### <font color="lime">**Creating Dataset Indicators for Collisions**</font>

Adding indicator fields to Collisions feature class

In [52]:
update_datetime("start")

print("Adding new fields to the Crashes table")
# Adding new fields to the Collisions table
arcpy.management.AddFields(
    in_table = os.path.join(workspace, "CollisionsPoints"),
    field_description = "CRASHES_INDICATOR LONG 'Crashes Dataset Indicator' # # #;PARTIES_INDICATOR LONG 'Parties Dataset Indicator' # # #;VICTIMS_INDICATOR LONG 'Victims Dataset Indicator' # # #",
    template = None
)
print(arcpy.GetMessages())

update_datetime("end")


Start: Wednesday November 13, 2024, 08:33 AM (PST)
Adding new fields to the Crashes table
Start Time: Wednesday, November 13, 2024 8:33:30 AM
Adding CRASHES_INDICATOR to CollisionsPoints...
Adding PARTIES_INDICATOR to CollisionsPoints...
Adding VICTIMS_INDICATOR to CollisionsPoints...
Succeeded at Wednesday, November 13, 2024 8:33:31 AM (Elapsed Time: 0.71 seconds)
End: Wednesday November 13, 2024, 08:33 AM (PST). Elapsed time: 0:00:01


#### <font color="deepskyblue">Calculating Victims indicator field</font>

- **Condition**: if VID is not empty, then set indicator value to 1, otherwise set indicator value to 0
- **Result**: gives all cases identifying unique victims records in the Collisions feature class

In [53]:
update_datetime("start")

print("Calculating new fields in the Crashes table")
# Calculate new fields in the Collisions table
arcpy.management.CalculateField(
    in_table = os.path.join(workspace, "CollisionsPoints"),
    field = "VICTIMS_INDICATOR",
    expression = "vind(!VID!)",
    expression_type = "PYTHON3",
    code_block = """def vind(x):
    if x != "":
        return 1
    else:
        return 0""",
    field_type = "TEXT",
    enforce_domains= "NO_ENFORCE_DOMAINS"
)
print(arcpy.GetMessages())

update_datetime("end")


Start: Wednesday November 13, 2024, 08:33 AM (PST)
Calculating new fields in the Crashes table
Start Time: Wednesday, November 13, 2024 8:33:36 AM
Succeeded at Wednesday, November 13, 2024 8:34:29 AM (Elapsed Time: 53.22 seconds)
End: Wednesday November 13, 2024, 08:34 AM (PST). Elapsed time: 0:00:53


#### <font color="deepskyblue">Calculating Parties indicator field</font>

- **Condition**: if Victim Number is 1 or None, set indicator value to 1, otherwise set indicator value to 0
- **Result**: gives all cases identifying unique parties records in the Collisions feature class

In [54]:
update_datetime("start")

print("Adding new fields to the Collisions table")
# Calculate new fields in the Collisions table
arcpy.management.CalculateField(
    in_table = os.path.join(workspace, "CollisionsPoints"),
    field = "PARTIES_INDICATOR",
    expression = "pind(!VICTIM_NUMBER!)",
    expression_type = "PYTHON3",
    code_block="""def pind(x):
    if x == 1 or x==None:
        return 1
    else:
        return 0""",
    field_type = "TEXT",
    enforce_domains= "NO_ENFORCE_DOMAINS"
)
print(arcpy.GetMessages())

update_datetime("end")


Start: Wednesday November 13, 2024, 08:34 AM (PST)
Adding new fields to the Collisions table
Start Time: Wednesday, November 13, 2024 8:34:29 AM
Succeeded at Wednesday, November 13, 2024 8:35:22 AM (Elapsed Time: 53.17 seconds)
End: Wednesday November 13, 2024, 08:35 AM (PST). Elapsed time: 0:00:53


#### <font color="deepskyblue">Calculating Crashes indicator field</font>

- **Condition**: if Party Number is 1 and Victim Number is either 1 or None, set indicator value to 1, otherwise set indicator value to 0
- **Result**: gives all cases identifying unique crashes records in the Collisions feature class

In [55]:
update_datetime("start")

print("Calculating new fields in the Collisions table")
# Calculate new fields in the Collisions table
arcpy.management.CalculateField(
    in_table = os.path.join(workspace, "CollisionsPoints"),
    field = "CRASHES_INDICATOR",
    expression = "cind(!PARTY_NUMBER!,!VICTIM_NUMBER!)",
    expression_type = "PYTHON3",
    code_block="""def cind(x,y):
    if x == 1:
        if y == 1 or y == None:
            return 1
    else:
        return 0""",
    field_type = "TEXT",
    enforce_domains= "NO_ENFORCE_DOMAINS"
)
print(arcpy.GetMessages())

update_datetime("end")


Start: Wednesday November 13, 2024, 08:35 AM (PST)
Calculating new fields in the Collisions table
Start Time: Wednesday, November 13, 2024 8:35:23 AM
Succeeded at Wednesday, November 13, 2024 8:36:16 AM (Elapsed Time: 53.62 seconds)
End: Wednesday November 13, 2024, 08:36 AM (PST). Elapsed time: 0:00:53


Update the field list and field mappings for the Collisions feature class

In [56]:
# Update fields in the Collisions table
fieldsCollisions = arcpy.ListFields(collisionsTable)

# Update the mappings for the Collisions table
mappingCollisions = get_field_mappings(codebook, fieldsCollisions, collisionsTable)


Save the ArcGIS Pro project

In [57]:
# Save the project
aprx.save()


### <font color="lime">**Obtaining data paths, locations and field tables**</font>

In this section we will obtain the paths, locations, and field tables for the feature classes.

Get a list of geodatabase tables

In [58]:
# List all tables in the geodatabase
gdbTables = arcpy.ListTables()
print(f"Geodatabase Tables: {gdbTables}")


Geodatabase Tables: ['CrashesTable', 'PartiesTable', 'VictimsTable', 'CollisionsTable']


Obtain the list of point feature classes from the geodatabase

In [59]:
# List all feature classes in the geodatabase root
gdbFeatureClasses = arcpy.ListFeatureClasses(feature_type="Point")
print(f"Geodatabase Point Feature Classes: {gdbFeatureClasses}")

# Get the feature classes by name
crashesPoints = arcpy.ListFeatureClasses("CrashesPoints")[0]
partiesPoints = arcpy.ListFeatureClasses("PartiesPoints")[0]
victimsPoints = arcpy.ListFeatureClasses("VictimsPoints")[0]
collisionsPoints = arcpy.ListFeatureClasses("CollisionsPoints")[0]


Geodatabase Point Feature Classes: ['CrashesPoints', 'PartiesPoints', 'VictimsPoints', 'CollisionsPoints']


Format the feature class paths for the four levels

In [60]:
# Get the paths of each of the layers: Crashes, Parties, Victims, Collisions
crashesPointsPath = os.path.join(workspace, crashesPoints)
partiesPointsPath = os.path.join(workspace, partiesPoints)
victimsPointsPath = os.path.join(workspace, victimsPoints)
collisionsPointsPath = os.path.join(workspace, collisionsPoints)

print(f"Feature Paths:\n- Crashes: {crashesPointsPath}\n- Parties: {partiesPointsPath}\n- Victims: {victimsPointsPath}\n- Collisions: {collisionsPointsPath}\n")


Feature Paths:
- Crashes: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\CrashesPoints
- Parties: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\PartiesPoints
- Victims: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\VictimsPoints
- Collisions: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\CollisionsPoints



Get the field lists for the four feature classes

In [61]:
# Get the field list for each of the feature classes
crashesPointsFields = arcpy.ListFields(crashesPointsPath)
partiesPointsFields = arcpy.ListFields(partiesPointsPath)
victimsPointsFields = arcpy.ListFields(victimsPointsPath)
collisionsPointsFields = arcpy.ListFields(collisionsPointsPath)


Get the row counts for the four feature classes

In [62]:
# Obtain the row counts for each of the feature classes
countCrashesPoints = int(arcpy.management.GetCount(crashesPointsPath)[0])
countPartiesPoints = int(arcpy.management.GetCount(partiesPointsPath)[0])
countVictimsPoints = int(arcpy.management.GetCount(victimsPointsPath)[0])
countCollisionsPoints = int(arcpy.management.GetCount(collisionsPointsPath)[0])

print(f"Point Feature Counts:\n- Crashes: {countCrashesPoints:,}\n- Parties: {countPartiesPoints:,}\n- Victims: {countVictimsPoints:,}\n- Collisions: {countCollisionsPoints:,}")


Point Feature Counts:
- Crashes: 149,764
- Parties: 323,542
- Victims: 260,120
- Collisions: 260,120


Update the field mappings for the four feature classes

In [63]:
mappingCrashesPoints = get_field_mappings(codebook, crashesPointsFields, crashesPointsPath)
mappingPartiesPoints = get_field_mappings(codebook, partiesPointsFields, partiesPointsPath)
mappingVictimsPoints = get_field_mappings(codebook, victimsPointsFields, victimsPointsPath)
mappingCollisionsPoints = get_field_mappings(codebook, collisionsPointsFields, collisionsPointsPath)


Modifying the field types for the four feature classes

In [64]:
print("Crashes: Modifying codebook field types...")
modify_agp_types(codebook, crashesPointsFields)


Crashes: Modifying codebook field types...


{'OBJECTID': {'no': 1,
  'level': 'GIS',
  'alias': 'Object ID',
  'desc': 'ESRI Object ID',
  'type': 'OID',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'GlobalID': {'no': 2,
  'level': 'GIS',
  'alias': 'Global ID',
  'desc': 'ESRI Global ID string identifier',
  'type': 'GlobalID',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'GUID': {'no': 3,
  'level': 'GIS',
  'alias': 'GUID Related Table Identifier',
  'desc': 'ESRI GUID to be used in merging related tables',
  'type': 'Guid',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'CASE_ID': {'no': 4,
  'level': 'Crash',
  'alias': 'Case ID',
  'desc': 'Unique identifier of the crash case',
  'type': 'LONG',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'CID': {'no': 5,
  'level': 'Crash',
  'alias': 'Crash ID',
  'desc': 'Unique identifier of the crash case',
  'type': 'TEXT',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'PID': {'no': 6,
  'level': 'Party',
  'alias': 'Party ID',
  'desc': 'Unique identifier for the party case'

In [65]:
print("Parties: Modifying codebook field types...")
modify_agp_types(codebook, partiesPointsFields)


Parties: Modifying codebook field types...


{'OBJECTID': {'no': 1,
  'level': 'GIS',
  'alias': 'Object ID',
  'desc': 'ESRI Object ID',
  'type': 'OID',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'GlobalID': {'no': 2,
  'level': 'GIS',
  'alias': 'Global ID',
  'desc': 'ESRI Global ID string identifier',
  'type': 'GlobalID',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'GUID': {'no': 3,
  'level': 'GIS',
  'alias': 'GUID Related Table Identifier',
  'desc': 'ESRI GUID to be used in merging related tables',
  'type': 'Guid',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'CASE_ID': {'no': 4,
  'level': 'Crash',
  'alias': 'Case ID',
  'desc': 'Unique identifier of the crash case',
  'type': 'LONG',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'CID': {'no': 5,
  'level': 'Crash',
  'alias': 'Crash ID',
  'desc': 'Unique identifier of the crash case',
  'type': 'TEXT',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'PID': {'no': 6,
  'level': 'Party',
  'alias': 'Party ID',
  'desc': 'Unique identifier for the party case'

In [66]:
print("Victims: Modifying codebook field types...")
modify_agp_types(codebook, victimsPointsFields)


Victims: Modifying codebook field types...


{'OBJECTID': {'no': 1,
  'level': 'GIS',
  'alias': 'Object ID',
  'desc': 'ESRI Object ID',
  'type': 'OID',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'GlobalID': {'no': 2,
  'level': 'GIS',
  'alias': 'Global ID',
  'desc': 'ESRI Global ID string identifier',
  'type': 'GlobalID',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'GUID': {'no': 3,
  'level': 'GIS',
  'alias': 'GUID Related Table Identifier',
  'desc': 'ESRI GUID to be used in merging related tables',
  'type': 'Guid',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'CASE_ID': {'no': 4,
  'level': 'Crash',
  'alias': 'Case ID',
  'desc': 'Unique identifier of the crash case',
  'type': 'LONG',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'CID': {'no': 5,
  'level': 'Crash',
  'alias': 'Crash ID',
  'desc': 'Unique identifier of the crash case',
  'type': 'TEXT',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'PID': {'no': 6,
  'level': 'Party',
  'alias': 'Party ID',
  'desc': 'Unique identifier for the party case'

In [67]:
print("Collisions: Modifying codebook field types...")
modify_agp_types(codebook, collisionsPointsFields)


Collisions: Modifying codebook field types...


{'OBJECTID': {'no': 1,
  'level': 'GIS',
  'alias': 'Object ID',
  'desc': 'ESRI Object ID',
  'type': 'OID',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'GlobalID': {'no': 2,
  'level': 'GIS',
  'alias': 'Global ID',
  'desc': 'ESRI Global ID string identifier',
  'type': 'GlobalID',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'GUID': {'no': 3,
  'level': 'GIS',
  'alias': 'GUID Related Table Identifier',
  'desc': 'ESRI GUID to be used in merging related tables',
  'type': 'Guid',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'CASE_ID': {'no': 4,
  'level': 'Crash',
  'alias': 'Case ID',
  'desc': 'Unique identifier of the crash case',
  'type': 'LONG',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'CID': {'no': 5,
  'level': 'Crash',
  'alias': 'Crash ID',
  'desc': 'Unique identifier of the crash case',
  'type': 'TEXT',
  'cat': 'Unique Identifier',
  'dbin': 'N'},
 'PID': {'no': 6,
  'level': 'Party',
  'alias': 'Party ID',
  'desc': 'Unique identifier for the party case'

Save the ArcGIS Pro project

In [68]:
# Save the project
aprx.save()


## <font color="orangered">**Assigning Domain Attributes**</font>

Creating and assigning domain attributes for the feature classes using the JSON codebook dictionary

### <font color="lime">**Creating domains in geodatabase**</font>

> ⚠️ <font color="coral">**Note**</font>: This command can only be run as and if it is necessary. Running this command will delete all domains and domain variables from the geodatabase.

Delete all domains from the geodatabase

In [69]:
# Delete domains from the geodatabase
for key in list(d.name for d in list(arcpy.da.ListDomains())):
    arcpy.management.DeleteDomain(workspace, key)
    print(f"{key}: Domain deleted")
    print(arcpy.GetMessages())


COLLISION_MONTH: Domain deleted
Start Time: Wednesday, November 13, 2024 8:37:14 AM
Succeeded at Wednesday, November 13, 2024 8:37:14 AM (Elapsed Time: 0.40 seconds)
COLLISION_TIME_INTERVALS: Domain deleted
Start Time: Wednesday, November 13, 2024 8:37:15 AM
Succeeded at Wednesday, November 13, 2024 8:37:15 AM (Elapsed Time: 0.38 seconds)
OAF_2: Domain deleted
Start Time: Wednesday, November 13, 2024 8:37:15 AM
Succeeded at Wednesday, November 13, 2024 8:37:15 AM (Elapsed Time: 0.35 seconds)
DAY_OF_WEEK: Domain deleted
Start Time: Wednesday, November 13, 2024 8:37:16 AM
Succeeded at Wednesday, November 13, 2024 8:37:16 AM (Elapsed Time: 0.39 seconds)
COLLISION_TIME_RUSH_HOURS: Domain deleted
Start Time: Wednesday, November 13, 2024 8:37:16 AM
Succeeded at Wednesday, November 13, 2024 8:37:17 AM (Elapsed Time: 0.39 seconds)
OAF_VIOL_CAT: Domain deleted
Start Time: Wednesday, November 13, 2024 8:37:17 AM
Succeeded at Wednesday, November 13, 2024 8:37:17 AM (Elapsed Time: 0.34 seconds)
PE

Define domain attributes and adding them to the geodatabase for all four levels. The list of domains and domain attributes are defined in the codebook JSON dictionary.

In [70]:
# Iterate through the keys of the codebook dictionary
for key in list(codebook.keys()):
    # Check if the key has a domain attribute
    if codebook[key]["dbin"] == "Y":
        # Check if the key is in the domain list of the geodatabase
        if key in list(d.name for d in list(arcpy.da.ListDomains())):
            # Let the user know that the domain already exists
            print(f"Match {codebook[key]['no']}: Domain {key} already exists. Aborting...")
        else:
            print(f"Match {codebook[key]['no']}: Domain {key} does not exist. Creating...")
            print(f"Creating domain for {key}...")
            # Create the domain in the geodatabase
            arcpy.management.CreateDomain(
                in_workspace = workspace,
                domain_name = key,
                domain_description = codebook[key]["desc"],
                field_type = codebook[key]["type"],
                domain_type = "CODED",
                split_policy = "DEFAULT",
                merge_policy = "DEFAULT"
            )
            print(arcpy.GetMessages())
            print(f"\tDefining coded values dictionary...")
            # Iterate through the domain values in the codebook dictionary
            for item in list(codebook[key]["domain"]):
                itemCode = codebook[key]["domain"][item]
                # Depending on the field type conditions:
                if codebook[key]["type"] in ["TEXT"]:
                    print(f"\t\t...adding {item}: {itemCode}")
                    # Add the domain value to the domain
                    arcpy.management.AddCodedValueToDomain(
                        in_workspace = workspace,
                        domain_name = key,
                        code = item,
                        code_description = itemCode
                    )
                elif codebook[key]["type"] in ["LONG", "SHORT", "DOUBLE"]:
                    if item.isnumeric():
                        print(f"\t\t...adding {item}: {itemCode}")
                        # Add the domain value to the domain
                        arcpy.management.AddCodedValueToDomain(
                            in_workspace = workspace,
                            domain_name = key,
                            code = int(item),
                            code_description = itemCode
                        )
                

Match 20: Domain COLLISION_MONTH does not exist. Creating...
Creating domain for COLLISION_MONTH...
Start Time: Wednesday, November 13, 2024 8:37:59 AM
Succeeded at Wednesday, November 13, 2024 8:37:59 AM (Elapsed Time: 0.29 seconds)
	Defining coded values dictionary...
		...adding 1: January
		...adding 2: February
		...adding 3: March
		...adding 4: April
		...adding 5: May
		...adding 6: June
		...adding 7: July
		...adding 8: August
		...adding 9: September
		...adding 10: October
		...adding 11: November
		...adding 12: December
Match 21: Domain DAY_OF_WEEK does not exist. Creating...
Creating domain for DAY_OF_WEEK...
Start Time: Wednesday, November 13, 2024 8:38:05 AM
Succeeded at Wednesday, November 13, 2024 8:38:05 AM (Elapsed Time: 0.29 seconds)
	Defining coded values dictionary...
		...adding 1: Monday
		...adding 2: Tuesday
		...adding 3: Wednesday
		...adding 4: Thursday
		...adding 5: Friday
		...adding 6: Saturday
		...adding 7: Sunday
Match 23: Domain COLLISION_TIME_INT

### <font color="lime">**Assigning domains to feature classes**</font>

#### <font color="deepskyblue">Crashes Feature Class</font>

Assigning domain attributes to fields for the Crashes feature class

In [71]:
update_datetime("start")

# Iterate through the fields of the Crashes feature class
for field in crashesPointsFields:
    # if both conditions are met
    if (field.name in list(codebook.keys())) and (codebook[field.name]["dbin"] == "Y"):
        # if, also, the field name is in the geodatabase domain list
        if field.name in list(d.name for d in list(arcpy.da.ListDomains())):
            print(f"Match {codebook[field.name]['no']}: Domain {field.name}")
            print(f"\tAssigning domain {field.name} to field.\n")
            # Assign the domain to the field
            arcpy.management.AssignDomainToField(
                in_table = crashesPointsPath,
                field_name = field.name,
                domain_name = field.name,
                subtype_code = None
            )
            
update_datetime("end")            


Start: Wednesday November 13, 2024, 08:46 AM (PST)
Match 20: Domain COLLISION_MONTH
	Assigning domain COLLISION_MONTH to field.

Match 21: Domain DAY_OF_WEEK
	Assigning domain DAY_OF_WEEK to field.

Match 23: Domain COLLISION_TIME_INTERVALS
	Assigning domain COLLISION_TIME_INTERVALS to field.

Match 24: Domain COLLISION_TIME_RUSH_HOURS
	Assigning domain COLLISION_TIME_RUSH_HOURS to field.

Match 25: Domain COLLISION_SEVERITY
	Assigning domain COLLISION_SEVERITY to field.

Match 26: Domain COLLISION_SEVERITY_BINARY
	Assigning domain COLLISION_SEVERITY_BINARY to field.

Match 27: Domain COLLISION_SEVERITY_RECLASS
	Assigning domain COLLISION_SEVERITY_RECLASS to field.

Match 28: Domain COLLISION_SEVERITY_RANKED
	Assigning domain COLLISION_SEVERITY_RANKED to field.

Match 42: Domain PRIMARY_COLL_FACTOR
	Assigning domain PRIMARY_COLL_FACTOR to field.

Match 43: Domain TYPE_OF_COLLISION
	Assigning domain TYPE_OF_COLLISION to field.

Match 44: Domain PEDESTRIAN_ACCIDENT
	Assigning domain PEDE

#### <font color="deepskyblue">Parties Feature Class</font>

Assigning domain attributes to fields for the Parties feature class

In [72]:
update_datetime("start")

# Iterate through the fields of the Parties feature class
for field in partiesPointsFields:
    # if both conditions are met
    if (field.name in list(codebook.keys())) and (codebook[field.name]["dbin"] == "Y"):
        # if, also, the field name is in the geodatabase domain list
        if field.name in list(d.name for d in list(arcpy.da.ListDomains())):
            print(f"Match {codebook[field.name]['no']}: Domain {field.name}")
            print(f"\tAssigning domain {field.name} to field.\n")
            # Assign the domain to the field
            arcpy.management.AssignDomainToField(
                in_table = partiesPointsPath,
                field_name = field.name,
                domain_name = field.name,
                subtype_code = None
            )
            
update_datetime("end")


Start: Wednesday November 13, 2024, 08:52 AM (PST)
Match 20: Domain COLLISION_MONTH
	Assigning domain COLLISION_MONTH to field.

Match 21: Domain DAY_OF_WEEK
	Assigning domain DAY_OF_WEEK to field.

Match 23: Domain COLLISION_TIME_INTERVALS
	Assigning domain COLLISION_TIME_INTERVALS to field.

Match 24: Domain COLLISION_TIME_RUSH_HOURS
	Assigning domain COLLISION_TIME_RUSH_HOURS to field.

Match 25: Domain COLLISION_SEVERITY
	Assigning domain COLLISION_SEVERITY to field.

Match 26: Domain COLLISION_SEVERITY_BINARY
	Assigning domain COLLISION_SEVERITY_BINARY to field.

Match 27: Domain COLLISION_SEVERITY_RECLASS
	Assigning domain COLLISION_SEVERITY_RECLASS to field.

Match 28: Domain COLLISION_SEVERITY_RANKED
	Assigning domain COLLISION_SEVERITY_RANKED to field.

Match 42: Domain PRIMARY_COLL_FACTOR
	Assigning domain PRIMARY_COLL_FACTOR to field.

Match 43: Domain TYPE_OF_COLLISION
	Assigning domain TYPE_OF_COLLISION to field.

Match 44: Domain PEDESTRIAN_ACCIDENT
	Assigning domain PEDE

#### <font color="deepskyblue">Victims Feature Class</font>

Assigning domain attributes to fields for the Victims feature class

In [73]:
update_datetime("start")

# Iterate through the fields of the Victims feature class
for field in victimsPointsFields:
    # if both conditions are met
    if (field.name in list(codebook.keys())) and (codebook[field.name]["dbin"] == "Y"):
        # if, also, the field name is in the geodatabase domain list
        if field.name in list(d.name for d in list(arcpy.da.ListDomains())):
            print(f"Match {codebook[field.name]['no']}: Domain {field.name}")
            print(f"\tAssigning domain {field.name} to field.\n")
            # Assign the domain to the field
            arcpy.management.AssignDomainToField(
                in_table = victimsPointsPath,
                field_name = field.name,
                domain_name = field.name,
                subtype_code = None
            )
            
update_datetime("end")


Start: Wednesday November 13, 2024, 08:53 AM (PST)
Match 20: Domain COLLISION_MONTH
	Assigning domain COLLISION_MONTH to field.

Match 21: Domain DAY_OF_WEEK
	Assigning domain DAY_OF_WEEK to field.

Match 23: Domain COLLISION_TIME_INTERVALS
	Assigning domain COLLISION_TIME_INTERVALS to field.

Match 24: Domain COLLISION_TIME_RUSH_HOURS
	Assigning domain COLLISION_TIME_RUSH_HOURS to field.

Match 25: Domain COLLISION_SEVERITY
	Assigning domain COLLISION_SEVERITY to field.

Match 26: Domain COLLISION_SEVERITY_BINARY
	Assigning domain COLLISION_SEVERITY_BINARY to field.

Match 27: Domain COLLISION_SEVERITY_RECLASS
	Assigning domain COLLISION_SEVERITY_RECLASS to field.

Match 28: Domain COLLISION_SEVERITY_RANKED
	Assigning domain COLLISION_SEVERITY_RANKED to field.

Match 42: Domain PRIMARY_COLL_FACTOR
	Assigning domain PRIMARY_COLL_FACTOR to field.

Match 43: Domain TYPE_OF_COLLISION
	Assigning domain TYPE_OF_COLLISION to field.

Match 44: Domain PEDESTRIAN_ACCIDENT
	Assigning domain PEDE

#### <font color="deepskyblue">Collisions Feature Class</font>

Assigning domain attributes to fields for the Collisions feature class

In [74]:
update_datetime("start")

# Iterate through the fields of the Collisions feature class
for field in collisionsPointsFields:
    # if both conditions are met
    if (field.name in list(codebook.keys())) and (codebook[field.name]["dbin"] == "Y"):
        # if, also, the field name is in the geodatabase domain list
        if field.name in list(d.name for d in list(arcpy.da.ListDomains())):
            print(f"Match {codebook[field.name]['no']}: Domain {field.name}")
            print(f"\tAssigning domain {field.name} to field.\n")
            # Assign the domain to the field
            arcpy.management.AssignDomainToField(
                in_table = collisionsPointsPath,
                field_name = field.name,
                domain_name = field.name,
                subtype_code = None
            )
            
update_datetime("end")


Start: Wednesday November 13, 2024, 08:54 AM (PST)
Match 20: Domain COLLISION_MONTH
	Assigning domain COLLISION_MONTH to field.

Match 21: Domain DAY_OF_WEEK
	Assigning domain DAY_OF_WEEK to field.

Match 23: Domain COLLISION_TIME_INTERVALS
	Assigning domain COLLISION_TIME_INTERVALS to field.

Match 24: Domain COLLISION_TIME_RUSH_HOURS
	Assigning domain COLLISION_TIME_RUSH_HOURS to field.

Match 25: Domain COLLISION_SEVERITY
	Assigning domain COLLISION_SEVERITY to field.

Match 26: Domain COLLISION_SEVERITY_BINARY
	Assigning domain COLLISION_SEVERITY_BINARY to field.

Match 27: Domain COLLISION_SEVERITY_RECLASS
	Assigning domain COLLISION_SEVERITY_RECLASS to field.

Match 28: Domain COLLISION_SEVERITY_RANKED
	Assigning domain COLLISION_SEVERITY_RANKED to field.

Match 42: Domain PRIMARY_COLL_FACTOR
	Assigning domain PRIMARY_COLL_FACTOR to field.

Match 43: Domain TYPE_OF_COLLISION
	Assigning domain TYPE_OF_COLLISION to field.

Match 44: Domain PEDESTRIAN_ACCIDENT
	Assigning domain PEDE

Save the ArcGIS Pro project

In [75]:
# Save the project
aprx.save()


## <font color="orangered">**Geodatabase Metadata Operations**</font>

### <font color="lime">**Feature Class Aliases**</font>

Define name aliases for the feature classes (to be used both for defining feature class aliases, and for calling layer names in map contents)

In [76]:
# Define aliases for each of the feature classes
aliasCrashes = "OCSWITRS Crashes"
aliasParties = "OCSWITRS Parties"
aliasVictims = "OCSWITRS Victims"
aliasCollisions = "OCSWITRS Collisions"
aliasCities = "OCSWITRS Cities"
aliasRoads = "OCSWITRS Roads"


Assign aliases to feature classes

In [77]:
# Assign alias operations to each of the feature classes
arcpy.AlterAliasName(crashesPointsPath, aliasCrashes)
arcpy.AlterAliasName(partiesPointsPath, aliasParties)
arcpy.AlterAliasName(victimsPointsPath, aliasVictims)
arcpy.AlterAliasName(collisionsPointsPath, aliasCollisions)
arcpy.AlterAliasName(citiesPath, aliasCities)
arcpy.AlterAliasName(roadsPath, aliasRoads)


Save the ArcGIS Pro project

In [78]:
# Save the project
aprx.save()


### <font color="lime">**Crashes Metadata**</font>

Create a new *Crashes* metadata object

In [79]:
# Define key metadata attributes for the Crashes feature class
mdoCrashes = md.Metadata()
mdoCrashes.title = "OCSWITRS Crashes Points"
mdoCrashes.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoCrashes.summary = "Statewide Integrated Traffic Records System (SWITRS) Crash Data for Orange County, California (2013-2024)"
mdoCrashes.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on crashes</span><span> in Orange County, California for 2013-2024 (December 31, 2012 to June 30, 2024). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{lastUpdateDate}</b></span></p></div></div></div>"""
mdoCrashes.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoCrashes.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoCrashes.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"


Assign the *Crashes* metadata object's content to the feature layer

In [80]:
# Apply the metadata object to the Crashes feature class
mdCrashes = md.Metadata(crashesPointsPath)
if not mdCrashes.isReadOnly:
    mdCrashes.copy(mdoCrashes)
    mdCrashes.save()


### <font color="lime">**Parties Metadata**</font>

Create a new *Parties* metadata object

In [81]:
# Define key metadata attributes for the Parties feature class
mdoParties = md.Metadata()
mdoParties.title = "OCSWITRS Parties Points"
mdoParties.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Parties, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoParties.summary = "Statewide Integrated Traffic Records System (SWITRS) Incident-Involved Parties Data for Orange County, California (2013-2024)"
mdoParties.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on parties involved in crash incidents</span><span> in Orange County, California for 2013-2024 (December 31, 2012 to June 30, 2024). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{lastUpdateDate}</b></span></p></div></div></div>"""
mdoParties.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoParties.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoParties.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/1e07bb1002f9457fa6fd3540fdb08e29/data"


Assign the *Parties* metadata object's content to the feature layer

In [82]:
# Apply the metadata object to the Parties feature class
mdParties = md.Metadata(partiesPointsPath)
if not mdParties.isReadOnly:
    mdParties.copy(mdoParties)
    mdParties.save()


### <font color="lime">**Victims Metadata**</font>

Create a new *Victims* metadata object

In [83]:
# Define key metadata attributes for the Victims feature class
mdoVictims = md.Metadata()
mdoVictims.title = "OCSWITRS Victims Points"
mdoVictims.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Victims, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoVictims.summary = "Statewide Integrated Traffic Records System (SWITRS) Incident-Involved Victims Data for Orange County, California (2013-2024)"
mdoVictims.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">reports on victims/persons involved in crash incidents</span><span> in Orange County, California for 2013-2024 (December 31, 2012 to June 30, 2024). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{lastUpdateDate}</b></span></p></div></div></div>"""
mdoVictims.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoVictims.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoVictims.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/78682395df4744009c58625f1db0c25b/data"


Assign the *Victims* metadata object's content to the feature layer

In [84]:
# Apply the metadata object to the Victims feature class
mdVictims = md.Metadata(victimsPointsPath)
if not mdVictims.isReadOnly:
    mdVictims.copy(mdoVictims)
    mdVictims.save()


### <font color="lime">**Collisions Metadata**</font>

Create a new *Collisions* metadata object

In [85]:
mdyears = f"{rawDateStart.strftime('%Y')}-{rawDateEnd.strftime('%Y')}"

# write string rawDateStart in format mont, date, year
mddates = f"{rawDateStart.strftime('%B %d, %Y')} to {rawDateEnd.strftime('%B %d, %Y')}"


In [86]:
# Define key metadata attributes for the Collisions feature class
mdoCollisions = md.Metadata()
mdoCollisions.title = "OCSWITRS Combined Collisions Points"
mdoCollisions.tags = "Orange County, California, Traffic, Traffic Conditions, Crashes, Collisions, Road Safety, Accidents, SWITRS, OCSWITRS, Transportation"
mdoCollisions.summary = f"Statewide Integrated Traffic Records System (SWITRS) Combined Collisions Data for Orange County, California ({mdyears})"
mdoCollisions.description = f"""<div style="text-align:Left;"><div><div><p><span style="font-weight:bold;">Statewide Integrated Traffic Records System (SWITRS)</span><span> location point data, containing </span><span style="font-weight:bold;">combined reports on collision crashes, parties, and victims</span><span> in Orange County, California for {mdyears} ({mddates}). The data are collected and maintained by the </span><a href="https://www.chp.ca.gov:443/" style="text-decoration:underline;"><span>California Highway Patrol (CHP)</span></a><span>, from incidents reported by local and government agencies. Original tabular datasets are provided by the </span><a href="https://tims.berkeley.edu:443/" style="text-decoration:underline;"><span>Transportation Injury Mapping System (TIMS)</span></a><span>. Only records with reported locational GPS attributes in Orange County are included in the spatial database (either from X and Y geocoded coordinates, or the longitude and latitude coordinates generated by the CHP officer on site). Incidents without valid coordinates are omitted from this spatial dataset representation. Last Updated on <b>{lastUpdateDate}</b></span></p></div></div></div>"""
mdoCollisions.credits = "Dr. Kostas Alexandridis, GISP, Data Scientist, OC Public Works, OC Survey Geospatial Services"
mdoCollisions.accessConstraints = """<div style="text-align:Left;"><p><span>The SWITRS data displayed are provided by the California Highway Patrol (CHP) reports through the Transportation Injury Mapping System (TIMS) of the University of California, Berkeley. Issues of report accuracy should be addressed to CHP.</span></p><p>The displayed mapped data can be used under a <a href="https://creativecommons.org/licenses/by-sa/3.0/" target="_blank">Creative Commons CC-SA-BY</a> License, providing attribution to TIMS, CHP, and OC Public Works, OC Survey Geospatial Services. </p><div>We make every effort to provide the most accurate and up-to-date data and information. Nevertheless, the data feed is provided, 'as is' and OC Public Work's standard <a href="https://www.ocgov.com/contact-county/disclaimer" target="_blank">Disclaimer</a> applies.<br /></div><div><br /></div><div>For any inquiries, suggestions or questions, please contact:</div><div><br /></div><div style="text-align:center;"><a href="https://www.linkedin.com/in/ktalexan/" target="_blank"><b>Dr. Kostas Alexandridis, GISP</b></a><br /></div><div style="text-align:center;">GIS Analyst | Spatial Complex Systems Scientist</div><div style="text-align:center;">OC Public Works/OC Survey Geospatial Applications</div><div style="text-align:center;"><div>601 N. Ross Street, P.O. Box 4048, Santa Ana, CA 92701</div><div>Email: <a href="mailto:kostas.alexandridis@ocpw.ocgov.com" target="_blank">kostas.alexandridis@ocpw.ocgov.com</a> | Phone: (714) 967-0826</div><div><br /></div></div></div>"""
mdoCollisions.thumbnailUri = "https://ocpw.maps.arcgis.com/sharing/rest/content/items/6b96b7d6d5394cbb95aa2fae390503a9/data"


Assign the *Collisions* metadata object's content to the feature layer

In [87]:
# Apply the metadata object to the Collisions feature class
mdCollisions = md.Metadata(collisionsPointsPath)
if not mdCollisions.isReadOnly:
    mdCollisions.copy(mdoCollisions)
    mdCollisions.save()


Save the ArcGIS Pro project

In [88]:
# Save the project
aprx.save()


## <font color="orangered">**Map Layer Operations**</font>

In this section we will be creating map layers for the feature classes.

### <font color="lime">**Visual Layer Operations**</font>

Add and rename layers in contents view, using their aliases

In [89]:
for lyr in map.listLayers():
    if "OCSWITRS" in lyr.name:
        map.removeLayer(lyr)


In [90]:
# Add the four feature classes (Crashes Points, Parties Points, Victims Points, Collisions Points) from the geodatabase to the map
crashesLyrName = map.addDataFromPath(crashesPointsPath)
partiesLyrName =map.addDataFromPath(partiesPointsPath)
victimsLyrName = map.addDataFromPath(victimsPointsPath)
collisionsLyrName = map.addDataFromPath(collisionsPointsPath)

# Add the supporting data Cities and Roads layers to the map
citiesLyrName = map.addDataFromPath(citiesPath)
roadsLyrName = map.addDataFromPath(roadsPath)

print(f"Added layers to the map:\n\t- {crashesLyrName}\n\t- {partiesLyrName}\n\t- {victimsLyrName}\n\t- {collisionsLyrName}\n\t- {citiesLyrName}\n\t- {roadsLyrName}")


Added layers to the map:
	- OCSWITRS Crashes
	- OCSWITRS Parties
	- OCSWITRS Victims
	- OCSWITRS Collisions
	- OCSWITRS Cities
	- OCSWITRS Roads


Define layers in current map

In [91]:
# Store the current list of layers to a new variable
mapLyrs = map.listLayers()
# Store the names of the layers in a list
mapLyrsList = [mlyr.name for mlyr in mapLyrs]

print("Active Map Layers:")
# Loop through the list of layers in the map, and print their names
for mlyr in mapLyrs:
    print(f" - {mlyr.name}")


Active Map Layers:
 - OCSWITRS Collisions
 - OCSWITRS Victims
 - OCSWITRS Parties
 - OCSWITRS Crashes
 - OCSWITRS Roads
 - OCSWITRS Cities
 - World Topographic Map


Define individual layers for each of the SWITRS feature classes

In [92]:
# Identify each of the Crashes, Parties, Victim and Collisions layers
crashesLyr = next((mlyr for mlyr in mapLyrs if mlyr.name == aliasCrashes), False)
partiesLyr = next((mlyr for mlyr in mapLyrs if mlyr.name == aliasParties), False)
victimsLyr = next((mlyr for mlyr in mapLyrs if mlyr.name == aliasVictims), False)
collisionsLyr = next((mlyr for mlyr in mapLyrs if mlyr.name == aliasCollisions), False)
citiesLyr = next((mlyr for mlyr in mapLyrs if mlyr.name == aliasCities), False)
roadsLyr = next((mlyr for mlyr in mapLyrs if mlyr.name == aliasRoads), False)

# Define a list of the four layers, and a list of supporting data layers
dataLyrsList = [crashesLyr, partiesLyr, victimsLyr, collisionsLyr]
suppLyrsList = [citiesLyr, roadsLyr]


Turn off the initial visibility of all layers in the map

In [93]:
# Iterate through the data layers list
for lyr in dataLyrsList:
    # If the layer is not False
    if lyr:
        # Set the visibility of the layer to True
        lyr.visible = False
        
# Iterate through the supporting data layers list
for lyr in suppLyrsList:
    # If the layer is not False
    if lyr:
        # Set the visibility of the layer to True
        lyr.visible = False


Save the ArcGIS Pro project

In [94]:
# Save the project
aprx.save()


### <font color="lime">**Map Layer Time Settings Configuration**</font>

In this subsection we will configure the time settings for the map layers, so that the map view will be time-enabled

> <font color="orange">⚠️ **Reminder**:</font> When doing an update on the raw data, make sure you update the the end-time date for the project (this changes with each update).
- Current End Date: 6/30/2024, thus the time settings for the map layers will be set to the end of this date (6/30/2024, 23:59:59, or better, 7/1/2024, 00:00:00).

Set time settings configuration for the map layers

In [95]:
# Define the key time parameters for the layers using a dictionary
timeSettings = {
    "st": datetime(2013, 1, 1, 0, 0),
    "et": datetime(2024, 7, 1, 0, 0),
    "td": datetime(2024, 7, 1, 0, 0) - datetime(2013, 1, 1, 0, 0),
    "stf": "COLLISION_DATETIME",
    "tsi": 1.0,
    "tsiu": "months",
    "tz": arcpy.mp.ListTimeZones("*Pacific*")[0]
} # where, st: start time, et: end time, td: time extent, stf: time field, tsi: time interval, tsiu: time units, tz: time zone


Enable layer time, if needed

In [96]:
# Iterate through the data layer list and enable time
for mylyr in dataLyrsList:
    # if the layer is not time-enabled
    if not mylyr.isTimeEnabled:
        # Enable time for the layer
        mylyr.enableTime("COLLISION_DATETIME", "", "TRUE", None)
    # Set the start time for the layer
    mylyr.time.startTime = timeSettings["st"]
    # Set the end time for the layer
    mylyr.time.endTime = timeSettings["et"]
    # Set the time field for the layer
    mylyr.time.startTimeField = timeSettings["stf"]
    # Set the time step interval for the layer
    mylyr.time.timeStepInterval = timeSettings["tsi"]
    # Set the time step interval units for the layer
    mylyr.time.timeStepIntervalUnits = timeSettings["tsiu"]
    # Set the time zone for the layer
    mylyr.time.timeZone = timeSettings["tz"]


Reset time step interval parameters (seems to be a problem showing 10 months instead of 1 month)

In [97]:
# Iterate through the data layer list and manually change the time's time step interval
for mylyr in dataLyrsList:
    if mylyr.isTimeEnabled:
        mylyr.time.timeStepInterval = 1.0


Read and display the current time settings of the layers

In [98]:
# Iterate through the data layer list and display the time settings
for mylyr in dataLyrsList:
    print(mylyr.name)    
    print(f" - Time Field: {codebook[timeSettings['stf']]['alias']} ({timeSettings['stf']})")
    print(f" - Time Step Intervals: {int(mylyr.time.timeStepInterval)} {mylyr.time.timeStepIntervalUnits}")
    print(f" - Start Time: {str(timeSettings['st'].strftime('%m/%d/%Y %H:%M %p'))}")
    print(f" - End Time: {str(timeSettings['et'].strftime('%m/%d/%Y %H:%M %p'))}")
    print(f" - Time Extent: {int(timeSettings['td'].days):,} days ({int((timeSettings['td'].days/365)*12)} months)")
    print(f" - Time Zone: {timeSettings['tz']}")


OCSWITRS Crashes
 - Time Field: Crash Date and Time (COLLISION_DATETIME)
 - Time Step Intervals: 1 months
 - Start Time: 01/01/2013 00:00 AM
 - End Time: 07/01/2024 00:00 AM
 - Time Extent: 4,199 days (138 months)
 - Time Zone: (UTC-08:00) Pacific Time (US & Canada)
OCSWITRS Parties
 - Time Field: Crash Date and Time (COLLISION_DATETIME)
 - Time Step Intervals: 1 months
 - Start Time: 01/01/2013 00:00 AM
 - End Time: 07/01/2024 00:00 AM
 - Time Extent: 4,199 days (138 months)
 - Time Zone: (UTC-08:00) Pacific Time (US & Canada)
OCSWITRS Victims
 - Time Field: Crash Date and Time (COLLISION_DATETIME)
 - Time Step Intervals: 1 months
 - Start Time: 01/01/2013 00:00 AM
 - End Time: 07/01/2024 00:00 AM
 - Time Extent: 4,199 days (138 months)
 - Time Zone: (UTC-08:00) Pacific Time (US & Canada)
OCSWITRS Collisions
 - Time Field: Crash Date and Time (COLLISION_DATETIME)
 - Time Step Intervals: 1 months
 - Start Time: 01/01/2013 00:00 AM
 - End Time: 07/01/2024 00:00 AM
 - Time Extent: 4,199 

Save the ArcGIS Pro project

In [99]:
# Save the project
aprx.save()


### <font color="lime">**Update Layer Views and File Lists**</font>

List project and directories, layers in contents view, and layer files in directories

In [101]:
# Display the project details
print(f"Project:\n\tName: {aprx}\n\tPath: {aprxPath}\n\tLayers Directory: {layersPath}")
# Display the layers in content view
print(f"Layers in Content View:\n\tCrashes: {crashesLyr}\n\tParties: {partiesLyr}\n\tVictims: {victimsLyr}\n\tCollisions: {collisionsLyr}\n\tCities: {citiesLyr}\n\tRoads: {roadsLyr}")
# Layers in Directory
print(f"Layers in Directory:\n\tCrashes: {crashesPointsPath}\n\tParties: {partiesPointsPath}\n\tVictims: {victimsPointsPath}\n\tCollisions: {collisionsPointsPath}\n\tCities: {citiesPath}\n\tRoads: {roadsPath}")


Project:
	Name: <arcpy._mp.ArcGISProject object at 0x000002AFBF334850>
	Path: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.aprx
	Layers Directory: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\Layers
Layers in Content View:
	Crashes: OCSWITRS Crashes
	Parties: OCSWITRS Parties
	Victims: OCSWITRS Victims
	Collisions: OCSWITRS Collisions
	Cities: OCSWITRS Cities
	Roads: OCSWITRS Roads
Layers in Directory:
	Crashes: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\CrashesPoints
	Parties: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\PartiesPoints
	Victims: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\VictimsPoints
	Collisions: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\CollisionsPoints
	Cities: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AGPSWITRS\AGPSWITRS.gdb\SupportingData\OCSWITRS_Cities
	Roads: I:\Professional\Projects-OCPW\OCTraffic\OCSWITRS\AG

## <font color="orangered">**Symbology Operations**</font>

### <font color="lime">**Data Layer Symbologies**</font>

Define symbology for each of the data layers (Crashes, Parties, Victims, and Collisions). The symbology is pre-saved in the SWITRS layer file, stored in the project's layer folder.

In [102]:
# Apply the symbology from the OCSWITRS layer file to the data layers
for layer in [crashesLyr, partiesLyr, victimsLyr, collisionsLyr]:
    # Set the layer's symbology to the OCSWITRS layer file
    arcpy.management.ApplySymbologyFromLayer(
        in_layer = layer,
        in_symbology_layer = os.path.join(layersPath, "OCSWITRS.lyrx"),
        symbology_fields = "VALUE_FIELD COLLISION_SEVERITY COLLISION_SEVERITY",
        update_symbology = "MAINTAIN"
    )


Define symbology for the supporting data layers (Cities, Roads). The symbology for each of the supporting data layers is pre-saved in the CITIES and ROADS layer files respectively, stored in the project's layer folder.

In [103]:
# Apply the symbology from the CITIES layer to the Cities data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = citiesLyr,
    in_symbology_layer = os.path.join(layersPath, "CITIES.lyrx"),
    symbology_fields = "VALUE_FIELD POPDENS POPDENS",
    update_symbology = "MAINTAIN"
)

# Apply the symbology from the ROADS layer to the Roads data layer
arcpy.management.ApplySymbologyFromLayer(
    in_layer = roadsLyr,
    in_symbology_layer = os.path.join(layersPath, "ROADS.lyrx"),
    symbology_fields = "VALUE_FIELD ROADCAT ROADCAT",
    update_symbology = "MAINTAIN"
)


<Result 'GPLYR_{2BD1CFC9-3A6D-4D47-915B-05C3CBB046BE}'>

Save the ArcGIS Pro project

In [104]:
# Save the project
aprx.save()


### <font color="lime">**Export Layer Views to Files**</font>

In this final section of the notebook, we will export the layer views to files. for the feature classes and supporting data layers.

Define layer paths and names for the layers to be created and exported to filesystem

In [105]:
# Get locations of the layers in the map
crashesLyrFile = os.path.join(layersPath, crashesLyr.name + ".lyrx")
partiesLyrFile = os.path.join(layersPath, partiesLyr.name + ".lyrx")
victimsLyrFile = os.path.join(layersPath, victimsLyr.name + ".lyrx")
collisionsLyrFile = os.path.join(layersPath, collisionsLyr.name + ".lyrx")
citiesLyrFile = os.path.join(layersPath, citiesLyr.name + ".lyrx")
roadsLyrFile = os.path.join(layersPath, roadsLyr.name + ".lyrx")


Save each of the layers to the filesystem

In [106]:
update_datetime("start")

# Save the Crashes layer to a layer file
arcpy.management.SaveToLayerFile(crashesLyr, crashesLyrFile, "ABSOLUTE")
print(arcpy.GetMessages())

# Save the Parties layer to a layer file
arcpy.management.SaveToLayerFile(partiesLyr, partiesLyrFile, "ABSOLUTE")
print(arcpy.GetMessages())

# Save the Victims layer to a layer file
arcpy.management.SaveToLayerFile(victimsLyr, victimsLyrFile, "ABSOLUTE")
print(arcpy.GetMessages())

# Save the Collisions layer to a layer file
arcpy.management.SaveToLayerFile(collisionsLyr, collisionsLyrFile, "ABSOLUTE")
print(arcpy.GetMessages())

# Save the Cities layer to a layer file
arcpy.management.SaveToLayerFile(citiesLyr, citiesLyrFile, "ABSOLUTE")
print(arcpy.GetMessages())

# Save the Roads layer to a layer file
arcpy.management.SaveToLayerFile(roadsLyr, roadsLyrFile, "ABSOLUTE")
print(arcpy.GetMessages())

update_datetime("end")


Start: Wednesday November 13, 2024, 09:10 AM (PST)
Start Time: Wednesday, November 13, 2024 9:10:01 AM
Succeeded at Wednesday, November 13, 2024 9:10:01 AM (Elapsed Time: 0.10 seconds)
Start Time: Wednesday, November 13, 2024 9:10:01 AM
Succeeded at Wednesday, November 13, 2024 9:10:01 AM (Elapsed Time: 0.08 seconds)
Start Time: Wednesday, November 13, 2024 9:10:01 AM
Succeeded at Wednesday, November 13, 2024 9:10:01 AM (Elapsed Time: 0.07 seconds)
Start Time: Wednesday, November 13, 2024 9:10:01 AM
Succeeded at Wednesday, November 13, 2024 9:10:02 AM (Elapsed Time: 0.09 seconds)
Start Time: Wednesday, November 13, 2024 9:10:02 AM
Succeeded at Wednesday, November 13, 2024 9:10:02 AM (Elapsed Time: 0.05 seconds)
Start Time: Wednesday, November 13, 2024 9:10:02 AM
Succeeded at Wednesday, November 13, 2024 9:10:02 AM (Elapsed Time: 0.04 seconds)
End: Wednesday November 13, 2024, 09:10 AM (PST). Elapsed time: 0:00:01


Save the ArcGIS Pro project

In [107]:
# Save the project
aprx.save()


### <font color="lime">**Pop-Ups Configuration (Arcade)**</font>

Run these code segments below after opening the ArcGIS Pro project to configure the pop-ups for the feature classes.

**<font color="orange">Arcade Expression:</font> Configure Pop-Ups for the Crashes layer**

- Language: `Arcade`
- Name: `expression0`
- Title: `popupCrashes`
- Expression:
    ```javascript
    var c = $feature.CID
    var ct = $feature.CITY
    var dt = Text($feature.COLLISION_DATETIME, 'M/D/YYYY H:m A')
    "Crash ID: " + c + " | Date: " + dt + ", " + ct
    ```
    

**<font color="orange">Arcade Expression:</font> Configure Pop-Ups for the Parties layer**

- Language: `Arcade`
- Name: `expression0`
- Title: `popupParties`
- Expression:
    ```javascript
    var c = $feature.CID
    var p = $feature.PARTY_NUMBER
    var ct = $feature.CITY
    var dt = Text($feature.COLLISION_DATETIME, 'M/D/YYYY H:m A')
    "Crash ID: " + c + ", Party: " + p + " | Date: " + dt + ", " + ct
    ```


**<font color="orange">Arcade Expression</font> Configure Pop-Ups for the Victims layer**

- Language: `Arcade`
- Name: `expression0`
- Title: `popupVictims`
- Expression:
    ```javascript
    var c = $feature.CID
    var p = $feature.PARTY_NUMBER
    var v = $feature.VICTIM_NUMBER
    var ct = $feature.CITY
    var dt = Text($feature.COLLISION_DATETIME, 'M/D/YYYY H:m A')
    "Crash ID: " + c + ", Party: " + p + ", Victim: " + v + " | Date: " + dt + ", " + ct
    ```

**<font color="orange">Arcade Expression</font> Configure Pop-Ups for the Collisions layer**

- Language: `Arcade`
- Name: `expression0`
- Title: `popupCollisions`
- Expression:
    ```javascript
    var c = $feature.CID
    var p = $feature.PARTY_NUMBER
    var v = $feature.VICTIM_NUMBER
    var ct = $feature.CITY
    var dt = Text($feature.COLLISION_DATETIME, 'M/D/YYYY H:m A')
    var exp1 = "Crash ID: " + c + ", Party: " + p + ", Victim: " + v + " | Date: " + dt + ", " + ct
    var exp2 = "Crash ID: " + c + ", Party: " + p + ", No Victim | Date: " + dt + ", " + ct
    if(!isEmpty(p) && !isEmpty(v)){
        return exp1
    } else if(!isEmpty(p) && isEmpty(v)){
        return exp2
    }
    ```
    

----

> <font color="coral">💬 **Note**:</font> <font color="lime"> This is the end of data pre-processing (Notebook Part 2). The ArcGIS Online (AGO) processing of the data is continuing on a new ArcGIS notebook (Part 3)</font>